In [1]:
# -*- coding: utf-8 -*-
"""
2022006_5_complete_ransac_p2plane.py

Complete script for Point Cloud Registration using RANSAC + Point-to-Plane ICP.
Processes a sequence of point clouds to estimate trajectory and build a global map.
"""

import numpy as np
import open3d as o3d
import copy
import os
from scipy.stats import ortho_group
import pandas as pd
import matplotlib.pyplot as plt
# from tabulate import tabulate # Only needed if running the separate experimental part

# === Core Helper Functions ===

def load_point_cloud(file_path):
    """Load a point cloud from file."""
    try:
        pcd = o3d.io.read_point_cloud(file_path)
        if not pcd.has_points():
            print(f"Warning: No points found in {file_path}")
            return None
        return pcd
    except Exception as e:
        print(f"Error loading point cloud {file_path}: {e}")
        return None

def visualize_point_clouds(source, target, source_transformed=None, window_name="Point Cloud Alignment"):
    """Visualize original source (red), target (green), and optionally transformed source (blue)."""
    vis_list = []

    if source is not None and source.has_points():
        source_temp = copy.deepcopy(source)
        source_temp.paint_uniform_color([1, 0, 0])  # Red for source
        vis_list.append(source_temp)

    if target is not None and target.has_points():
        target_temp = copy.deepcopy(target)
        target_temp.paint_uniform_color([0, 1, 0])  # Green for target
        vis_list.append(target_temp)

    if source_transformed is not None and source_transformed.has_points():
        source_transformed_temp = copy.deepcopy(source_transformed)
        source_transformed_temp.paint_uniform_color([0, 0, 1])  # Blue for transformed source
        vis_list.append(source_transformed_temp)

    if vis_list:
        o3d.visualization.draw_geometries(vis_list, window_name=window_name)
    else:
        print("Warning: Nothing to visualize.")


def preprocess_point_cloud(pcd, voxel_size=0.05, estimate_normals=True, knn=30):
    """Preprocess point cloud: downsample and estimate normals."""
    if pcd is None:
        return None

    # Create a copy to avoid modifying the original
    processed_pcd = copy.deepcopy(pcd)

    # Downsample using voxel grid filter
    # print(f"    Downsampling with voxel size {voxel_size}...") # Verbose
    processed_pcd = processed_pcd.voxel_down_sample(voxel_size)
    if not processed_pcd.has_points():
        print("Warning: Point cloud has no points after downsampling.")
        return None

    # Estimate normals if requested
    if estimate_normals:
        if not processed_pcd.has_normals(): # Only estimate if not already present
             # print(f"    Estimating normals with KNN={knn}...") # Verbose
             search_param = o3d.geometry.KDTreeSearchParamKNN(knn=knn)
             processed_pcd.estimate_normals(search_param=search_param)
             # Orient normals consistently
             processed_pcd.orient_normals_consistent_tangent_plane(k=knn)
             if not processed_pcd.has_normals():
                  print("Warning: Failed to estimate normals.")

    return processed_pcd

def get_random_orthogonal_matrix():
    """Generate a random valid 4x4 transformation matrix."""
    R = ortho_group.rvs(3)
    t = np.random.rand(3, 1) * 0.1 # Small random translation

    initial_transform = np.eye(4)
    initial_transform[:3, :3] = R
    initial_transform[:3, 3] = t.squeeze()

    return initial_transform

def get_initial_transform_from_ransac(source, target, voxel_size, with_normals=True):
    """Use RANSAC for initial alignment (global registration)."""
    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Invalid input point clouds for RANSAC.")
        return np.eye(4)

    distance_threshold = voxel_size * 1.5 # Max correspondence distance for RANSAC

    if with_normals:
        if not source.has_normals() or not target.has_normals():
            print("Error: Normals required for feature matching RANSAC but not found.")
            return np.eye(4) # Return Identity if normals are required but missing

        # print("    Computing FPFH features...") # Verbose
        source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source,
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))
        target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target,
            o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

        # if source_fpfh is None or target_fpfh is None or not source_fpfh.has_data() or not target_fpfh.has_data():
        #      print("Error: FPFH feature computation failed or resulted in empty features.")
        #      return np.eye(4)

        # Check if feature computation was successful and resulted in actual data
        # The .data attribute is a NumPy array of shape (num_features, feature_dim)
        source_data_valid = source_fpfh is not None and source_fpfh.data.shape[0] > 0 and source_fpfh.data.shape[1] > 0
        target_data_valid = target_fpfh is not None and target_fpfh.data.shape[0] > 0 and target_fpfh.data.shape[1] > 0

        if not source_data_valid or not target_data_valid:
            print("Error: FPFH feature computation failed or resulted in empty features.")
            # Optional: Print shapes for debugging if needed
            # print(f"Source FPFH shape: {source_fpfh.data.shape if source_fpfh is not None else 'None'}")
            # print(f"Target FPFH shape: {target_fpfh.data.shape if target_fpfh is not None else 'None'}")
            return np.eye(4)

        # print("    Running RANSAC based on features...") # Verbose
        # Check Open3D version for mutual_filter compatibility if needed
        try:
            result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                source, target, source_fpfh, target_fpfh,
                mutual_filter=True, # Requires Open3D >= 0.13
                max_correspondence_distance=distance_threshold,
                estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # Point-to-Point for RANSAC transform estimation itself
                ransac_n=4, # Typically 3 or 4 points
                checkers=[
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
                ],
                # criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999) # Max iterations, confidence
                criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500) # Alternative higher params
            )
        except TypeError: # Handle potential absence of mutual_filter in older Open3D
             print("    Running RANSAC without mutual_filter (older Open3D version?).")
             result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                source, target, source_fpfh, target_fpfh,
                # mutual_filter=True, # Removed
                max_correspondence_distance=distance_threshold,
                estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
                ransac_n=4,
                checkers=[
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                    o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
                ],
                criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
            )
        except Exception as e:
             print(f"An unexpected error occurred during RANSAC: {e}")
             return np.eye(4)

        # Check if RANSAC result is valid (sometimes returns identity on failure)
        if np.allclose(result.transformation, np.identity(4)):
            print("Warning: RANSAC returned identity matrix. Initial guess might be poor.")

        return result.transformation
    else:
        # RANSAC based on correspondences (if pick_correspondences exists)
        print("Error: RANSAC without normals is not implemented/recommended here.")
        return np.eye(4)


def run_icp(source, target, initial_transform,
            max_iteration=100, threshold=0.05,
            knn=30, voxel_size=0.05, use_point_to_plane=True):
    """Run ICP (Point-to-Plane recommended) for refinement."""

    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Invalid input point clouds for ICP.")
        return np.eye(4), 0.0, 0.0, 0.0, 0.0 # Return identity and zero metrics on failure

    # Ensure point clouds have normals if using Point-to-Plane
    if use_point_to_plane:
        if not source.has_normals():
            print("    Estimating normals for source in ICP...")
            source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn))
            source.orient_normals_consistent_tangent_plane(k=knn)
        if not target.has_normals():
            print("    Estimating normals for target in ICP...")
            target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn))
            target.orient_normals_consistent_tangent_plane(k=knn)

        if not source.has_normals() or not target.has_normals():
             print("Warning: Failed to estimate normals for Point-to-Plane ICP. Falling back to Point-to-Point.")
             use_point_to_plane = False # Fallback

    if initial_transform is None:
        print("Warning: initial_transform is None in run_icp. Using identity.")
        initial_transform = np.eye(4) # Use identity, RANSAC should have provided one

    # Choose estimation method
    if use_point_to_plane:
        estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    else:
        estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    # Evaluate initial alignment (optional, for logging)
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, initial_transform)
    initial_fitness = evaluation.fitness
    initial_rmse = evaluation.inlier_rmse
    # print(f"    ICP Initial - Fitness: {initial_fitness:.4f}, RMSE: {initial_rmse:.4f}") # Verbose

    # Run ICP
    reg_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold, initial_transform,
        estimation_method,
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6, relative_rmse=1e-6, max_iteration=max_iteration)) # Added relative criteria

    # Get results
    transformation = reg_result.transformation
    fitness = reg_result.fitness
    inlier_rmse = reg_result.inlier_rmse

    return transformation, initial_fitness, initial_rmse, fitness, inlier_rmse


# === Sequence Registration Function (RANSAC + ICP) ===

def register_all_point_clouds_with_ransac(dataset_dir, pcd_files,
                                          ransac_voxel_size=0.05,
                                          icp_threshold=0.05,
                                          icp_max_iteration=200,
                                          knn=30):
    """
    Register all point clouds using RANSAC initial alignment + Point-to-Plane ICP refinement.
    """
    trajectory = []
    registered_pcds = [] # Stores original resolution clouds transformed to global frame
    registration_metrics = []

    # Load and prepare the first point cloud
    first_pcd_path = os.path.join(dataset_dir, pcd_files[0])
    first_pcd = load_point_cloud(first_pcd_path)
    if first_pcd is None:
        print("Error: Could not load the first point cloud. Aborting.")
        return None, None, None

    # Preprocess first cloud (downsample, estimate normals) - needed as target
    print(f"Preprocessing cloud 0 ({pcd_files[0]})...")
    first_pcd_processed = preprocess_point_cloud(first_pcd,
                                                 voxel_size=ransac_voxel_size,
                                                 estimate_normals=True, knn=knn)
    if first_pcd_processed is None:
        print("Error: Failed to preprocess the first point cloud. Aborting.")
        return None, None, None

    registered_pcds.append(copy.deepcopy(first_pcd)) # Store original high-res cloud
    global_transform = np.eye(4)
    trajectory.append(global_transform)

    prev_pcd_processed = first_pcd_processed # Keep track of the *processed* previous cloud
    last_successful_transform = np.eye(4) # Keep track of last good relative transform as fallback

    # Process subsequent pairs
    for i in range(1, len(pcd_files)):
        print("-" * 40)
        print(f"Processing pair {i-1}-{i} : ({pcd_files[i-1]} -> {pcd_files[i]})")
        print("-" * 40)

        # Initialize metrics for this pair (in case of early exit)
        current_metrics = {
            'pair': f"{i-1}-{i}", 'source': pcd_files[i], 'target': pcd_files[i-1],
            'ransac_fitness': -1, 'ransac_rmse': -1, 'initial_fitness_icp': -1,
            'initial_rmse_icp': -1, 'final_fitness': -1, 'final_rmse': -1,
        }
        relative_transform = np.eye(4) # Default to identity if anything fails

        # Load current point cloud (original resolution)
        current_pcd_path = os.path.join(dataset_dir, pcd_files[i])
        current_pcd = load_point_cloud(current_pcd_path)
        if current_pcd is None:
            print(f"Warning: Skipping pair {i-1}-{i} due to loading error for {pcd_files[i]}. Using identity transform.")
            # global_transform remains unchanged, trajectory appends same pose again? No, use last successful
            global_transform = global_transform @ last_successful_transform # Estimate based on last motion
            trajectory.append(copy.deepcopy(global_transform))
            registered_pcds.append(o3d.geometry.PointCloud()) # Append empty placeholder
            registration_metrics.append(current_metrics)
            continue # Skip to next iteration

        # Preprocess current point cloud for RANSAC and ICP
        print(f"Preprocessing cloud {i}...")
        current_pcd_processed = preprocess_point_cloud(current_pcd,
                                                      voxel_size=ransac_voxel_size,
                                                      estimate_normals=True, knn=knn)
        if current_pcd_processed is None:
             print(f"Warning: Skipping pair {i-1}-{i} due to preprocessing error for {pcd_files[i]}. Using identity transform.")
             global_transform = global_transform @ last_successful_transform # Estimate based on last motion
             trajectory.append(copy.deepcopy(global_transform))
             registered_pcds.append(o3d.geometry.PointCloud()) # Append empty placeholder
             registration_metrics.append(current_metrics)
             continue # Skip to next iteration

        # --- RANSAC for Initial Alignment ---
        print("Running RANSAC for initial alignment...")
        # Note: RANSAC estimates transform from source (current) to target (prev)
        initial_transform_ransac = get_initial_transform_from_ransac(
            current_pcd_processed, prev_pcd_processed, # Use processed clouds
            ransac_voxel_size,
            with_normals=True)

        # Evaluate RANSAC result (optional logging)
        eval_ransac = o3d.pipelines.registration.evaluate_registration(
            current_pcd_processed, prev_pcd_processed, icp_threshold, initial_transform_ransac) # Use ICP threshold for comparable eval
        current_metrics['ransac_fitness'] = eval_ransac.fitness
        current_metrics['ransac_rmse'] = eval_ransac.inlier_rmse
        print(f"RANSAC Initial Guess - Fitness: {eval_ransac.fitness:.4f}, RMSE: {eval_ransac.inlier_rmse:.4f}")

        ransac_failed = False
        if np.allclose(initial_transform_ransac, np.identity(4)) or eval_ransac.fitness < 0.01: # If RANSAC failed or found very few correspondences
            print("Warning: RANSAC failed or result is poor. ICP might struggle.")
            ransac_failed = True
            # We might still try ICP starting from identity or last known transform
            # initial_transform_ransac = last_successful_transform # Option: Use last motion as guess? Risky.
            initial_transform_ransac = np.identity(4) # Option: Start ICP from identity

        # --- Point-to-Plane ICP Refinement ---
        print("Running Point-to-Plane ICP for refinement...")
        # ICP refines the transform from source (current) to target (prev)
        transformation_icp, init_f, init_e, final_fitness, final_rmse = run_icp(
            current_pcd_processed, prev_pcd_processed,
            initial_transform=initial_transform_ransac,  # Use RANSAC result (or fallback)!
            max_iteration=icp_max_iteration,
            threshold=icp_threshold,
            use_point_to_plane=True, # Use Point-to-Plane
            knn=knn,
            voxel_size=ransac_voxel_size
        )
        current_metrics['initial_fitness_icp'] = init_f
        current_metrics['initial_rmse_icp'] = init_e
        current_metrics['final_fitness'] = final_fitness
        current_metrics['final_rmse'] = final_rmse

        # --- Calculate Relative Motion ---
        # We found T_curr_to_prev. Relative motion T_prev_to_curr is inv(T_curr_to_prev)
        icp_failed = False
        if final_fitness < 0.05 or final_rmse < 0 : # Check fitness and if RMSE is valid (non-negative)
             print(f"Warning: Low ICP fitness ({final_fitness:.3f}) or invalid RMSE ({final_rmse:.3f}) for pair {i-1}-{i}.")
             icp_failed = True
        elif final_rmse > icp_threshold * 5 : # If RMSE is much larger than threshold - likely failed alignment
            print(f"Warning: High ICP RMSE ({final_rmse:.3f}) for pair {i-1}-{i}.")
            icp_failed = True


        if icp_failed:
            print("ICP result is poor. Considering fallback.")
            if not ransac_failed and eval_ransac.fitness > 0.1: # If RANSAC was okay, maybe use its result
                 print("Using RANSAC transform directly due to poor ICP result.")
                 relative_transform = np.linalg.inv(initial_transform_ransac) # T_prev_to_curr
            else: # Both RANSAC and ICP seem bad, use last known motion
                 print("Both RANSAC and ICP seem poor. Using last successful relative transform.")
                 relative_transform = last_successful_transform
        else:
            # ICP result seems reasonable, use its inverse for relative motion
             relative_transform = np.linalg.inv(transformation_icp)
             last_successful_transform = relative_transform # Update last good transform


        # Update global transform and trajectory
        global_transform = global_transform @ relative_transform
        trajectory.append(copy.deepcopy(global_transform))

        # Transform the *original* high-resolution current point cloud to global frame
        transformed_pcd = copy.deepcopy(current_pcd)
        transformed_pcd.transform(global_transform)
        registered_pcds.append(transformed_pcd)

        # Store metrics for this pair
        registration_metrics.append(current_metrics)

        print(f"Registration {i}/{len(pcd_files)-1} - ICP Fitness: {final_fitness:.6f}, ICP RMSE: {final_rmse:.6f}")

        # Update the previous processed cloud for the next iteration's target
        prev_pcd_processed = current_pcd_processed

    metrics_df = pd.DataFrame(registration_metrics)
    return registered_pcds, trajectory, metrics_df

# === Visualization and Analysis Functions ===

def save_trajectory_to_csv(trajectory, output_path):
    """Save the trajectory (poses) to a CSV file."""
    positions = []
    for frame_idx, transform in enumerate(trajectory):
        x, y, z = transform[:3, 3]
        rotation = transform[:3, :3]

        # ZYX Euler angles (yaw, pitch, roll) - Handle potential gimbal lock issues if needed
        try:
            pitch = np.arcsin(-rotation[2, 0])
            if np.isclose(np.cos(pitch), 0.0): # Check for near gimbal lock
                # Gimbal lock: Set roll to 0 and calculate yaw relative to Z
                roll = 0.0
                yaw = np.arctan2(rotation[0, 1], rotation[1, 1]) # Use alternative formula
                # print(f"Warning: Potential gimbal lock detected at frame {frame_idx}. Roll set to 0.")
            else:
                roll = np.arctan2(rotation[2, 1], rotation[2, 2])
                yaw = np.arctan2(rotation[1, 0], rotation[0, 0])

            roll_deg, pitch_deg, yaw_deg = np.degrees(roll), np.degrees(pitch), np.degrees(yaw)

        except ValueError: # Handle potential math domain errors if rotation matrix is ill-conditioned
             print(f"Error calculating Euler angles for frame {frame_idx}. Setting to NaN.")
             roll_deg, pitch_deg, yaw_deg = np.nan, np.nan, np.nan

        positions.append({
            'x': x, 'y': y, 'z': z,
            'roll_deg': roll_deg, 'pitch_deg': pitch_deg, 'yaw_deg': yaw_deg
        })

    trajectory_df = pd.DataFrame(positions)
    try:
        trajectory_df.to_csv(output_path, index_label='frame')
        print(f"Trajectory saved to '{output_path}'")
    except Exception as e:
        print(f"Error saving trajectory to CSV: {e}")

    return trajectory_df

def plot_trajectory_3d(trajectory_df, filename='turtlebot_trajectory_3d.png'):
    """Plot the estimated 3D trajectory."""
    if trajectory_df is None or trajectory_df.empty:
        print("Cannot plot empty trajectory.")
        return None, None

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot(trajectory_df['x'], trajectory_df['y'], trajectory_df['z'], 'b-', linewidth=2, label='Robot Path')
    ax.scatter(trajectory_df['x'], trajectory_df['y'], trajectory_df['z'], c='red', s=30, label='Waypoints', alpha=0.7)
    ax.scatter(trajectory_df['x'].iloc[0], trajectory_df['y'].iloc[0], trajectory_df['z'].iloc[0],
               c='lime', s=100, label='Start', edgecolors='k')
    ax.scatter(trajectory_df['x'].iloc[-1], trajectory_df['y'].iloc[-1], trajectory_df['z'].iloc[-1],
               c='purple', s=100, label='End', edgecolors='k')

    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title('Estimated TurtleBot 3D Trajectory (RANSAC + P2Plane ICP)')

    # Make axes equal scale for better visualization
    try:
        x = trajectory_df['x'].dropna()
        y = trajectory_df['y'].dropna()
        z = trajectory_df['z'].dropna()
        if len(x) > 0 and len(y) > 0 and len(z) > 0: # Check if data exists after dropna
            max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
            if max_range == 0: max_range = 1.0 # Avoid division by zero or zero range

            mid_x = (x.max()+x.min()) * 0.5
            mid_y = (y.max()+y.min()) * 0.5
            mid_z = (z.max()+z.min()) * 0.5
            ax.set_xlim(mid_x - max_range, mid_x + max_range)
            ax.set_ylim(mid_y - max_range, mid_y + max_range)
            ax.set_zlim(mid_z - max_range, mid_z + max_range)
            ax.set_box_aspect([1,1,1]) # Attempt to make axes visually equal
        else:
            print("Not enough valid data points to set equal axes.")

    except Exception as e:
        print(f"Could not set equal axes: {e}")


    ax.legend()
    ax.grid(True)
    try:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"3D trajectory plot saved to '{filename}'")
    except Exception as e:
        print(f"Error saving 3D plot: {e}")
    plt.close(fig) # Close plot to free memory
    return fig, ax

def plot_trajectory_2d(trajectory_df, filename='turtlebot_trajectory_2d.png'):
    """Plot the estimated 2D (X-Y) trajectory projection."""
    if trajectory_df is None or trajectory_df.empty:
        print("Cannot plot empty trajectory.")
        return

    plt.figure(figsize=(10, 10))

    plt.plot(trajectory_df['x'], trajectory_df['y'], 'b-', linewidth=2, label='Robot Path')
    plt.scatter(trajectory_df['x'], trajectory_df['y'], c='red', s=30, label='Waypoints', alpha=0.7)
    plt.scatter(trajectory_df['x'].iloc[0], trajectory_df['y'].iloc[0], c='lime', s=100, label='Start', edgecolors='k')
    plt.scatter(trajectory_df['x'].iloc[-1], trajectory_df['y'].iloc[-1], c='purple', s=100, label='End', edgecolors='k')

    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title('Estimated TurtleBot 2D Trajectory (Top-Down View, RANSAC + P2Plane ICP)')
    plt.grid(True)
    plt.axis('equal') # Ensure aspect ratio is equal
    plt.legend()
    try:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"2D trajectory plot saved to '{filename}'")
    except Exception as e:
        print(f"Error saving 2D plot: {e}")
    plt.close() # Close plot


def visualize_combined_point_cloud(registered_pcds, trajectory=None, downsample_voxel_size=0.05, save_path="global_registered_pointcloud_ransac_p2plane.pcd", window_name="Combined Map"):
    """Visualize the combined registered point cloud map and optionally the trajectory."""
    if not registered_pcds:
        print("No registered point clouds to visualize.")
        return

    map_vis_elements = []
    combined_pcd_for_saving = o3d.geometry.PointCloud()

    # Add registered point clouds (colored uniquely)
    num_clouds = len(registered_pcds)
    for i, pcd in enumerate(registered_pcds):
        if pcd is None or not pcd.has_points():
            print(f"Skipping visualization/saving for empty cloud index {i}")
            continue

        # Downsample for visualization performance
        pcd_down = pcd.voxel_down_sample(voxel_size=downsample_voxel_size)
        if not pcd_down.has_points():
            continue

        pcd_colored = copy.deepcopy(pcd_down)

        # Assign distinct color using HSV space
        hue = i / float(num_clouds) if num_clouds > 0 else 0
        color = plt.cm.hsv(hue)[:3] # Get RGB from HSV colormap
        pcd_colored.paint_uniform_color(color)
        map_vis_elements.append(pcd_colored)

        # Add to combined cloud for saving (use downsampled version to keep file size manageable)
        combined_pcd_for_saving += pcd_down

    # Add trajectory visualization if provided
    if trajectory is not None and len(trajectory) > 1:
        line_set = o3d.geometry.LineSet()
        points = np.array([T[:3, 3] for T in trajectory])
        lines = [[i, i + 1] for i in range(len(points) - 1)]
        # Ensure lines only connect valid points if trajectory contains NaNs? For now assume valid.
        line_set.points = o3d.utility.Vector3dVector(points)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in range(len(lines))]) # Red trajectory
        map_vis_elements.append(line_set)

        # Add coordinate frames at key poses
        step = max(1, len(trajectory) // 10) # Show ~10 frames
        for i in range(0, len(trajectory), step):
             # Check if transform is valid before adding frame
            if np.isfinite(trajectory[i]).all():
                coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3, origin=[0, 0, 0])
                coord_frame.transform(trajectory[i])
                map_vis_elements.append(coord_frame)

    # Visualize
    if map_vis_elements:
        print("Visualizing combined map and trajectory...")
        o3d.visualization.draw_geometries(map_vis_elements, window_name=window_name)
    else:
        print("No elements to visualize.")

    # Save the combined point cloud
    if combined_pcd_for_saving.has_points():
        try:
            o3d.io.write_point_cloud(save_path, combined_pcd_for_saving)
            print(f"Saved global registered point cloud to '{save_path}'")
        except Exception as e:
            print(f"Error saving combined point cloud: {e}")
    else:
        print("No points in combined cloud to save.")


# === Main Execution Block ===

def main():
    # --- Configuration ---
    # IMPORTANT: Update this path to your dataset location!
    dataset_dir = "C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds"
    output_dir = "." # Directory to save outputs (CSV, PNG, PCD)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Registration Parameters
    ransac_voxel_size = 0.05    # Voxel size for preprocessing before RANSAC/ICP
    icp_threshold = 0.05        # Max correspondence distance for ICP refinement (meters)
    icp_max_iteration = 100     # Max iterations for ICP refinement (adjust based on convergence)
    knn_normals = 30            # K-neighbors for normal estimation

    # Visualization/Saving Parameters
    map_vis_voxel_size = 0.05   # Voxel size for downsampling the final map visualization/saving
    trajectory_csv_file = os.path.join(output_dir, "turtlebot_trajectory_ransac_p2plane.csv")
    metrics_csv_file = os.path.join(output_dir, "registration_metrics_ransac_p2plane.csv")
    plot_3d_file = os.path.join(output_dir, "turtlebot_trajectory_3d_ransac_p2plane.png")
    plot_2d_file = os.path.join(output_dir, "turtlebot_trajectory_2d_ransac_p2plane.png")
    global_map_file = os.path.join(output_dir, "global_registered_pointcloud_ransac_p2plane.pcd")
    summary_report_file = os.path.join(output_dir, "registration_summary_ransac_p2plane.txt")
    # --- End Configuration ---


    # Get sorted list of point cloud files
    try:
        all_files = os.listdir(dataset_dir)
    except FileNotFoundError:
        print(f"Error: Dataset directory not found: {dataset_dir}")
        print("Please update the 'dataset_dir' variable in the script.")
        return

    pcd_files = sorted([f for f in all_files if f.startswith("pointcloud_") and f.endswith(".pcd")])

    if not pcd_files:
        print(f"Error: No PCD files matching 'pointcloud_*.pcd' found in '{dataset_dir}'.")
        print("Check the path and file naming convention.")
        return

    print(f"Found {len(pcd_files)} point cloud files.")

    # --- Run Registration ---
    print("\nStarting registration process using RANSAC + Point-to-Plane ICP...")
    print(f"Parameters: RANSAC/ICP Voxel={ransac_voxel_size}, ICP Thresh={icp_threshold}, ICP Iter={icp_max_iteration}, Normals KNN={knn_normals}")

    registered_pcds, trajectory, metrics_df = register_all_point_clouds_with_ransac(
        dataset_dir, pcd_files,
        ransac_voxel_size=ransac_voxel_size,
        icp_threshold=icp_threshold,
        icp_max_iteration=icp_max_iteration,
        knn=knn_normals
    )

    if registered_pcds is None or trajectory is None or metrics_df is None:
        print("\nRegistration process failed or was aborted.")
        return

    print("\nRegistration processing complete!")
    # Filter out potential empty clouds added during errors
    valid_registered_pcds = [p for p in registered_pcds if p is not None and p.has_points()]
    print(f"Successfully processed {len(valid_registered_pcds)} point clouds.")
    print(f"Trajectory contains {len(trajectory)} poses.")

    # --- Save and Analyze Results ---
    if not metrics_df.empty:
        try:
            metrics_df.to_csv(metrics_csv_file, index=False)
            print(f"Saved registration metrics to '{metrics_csv_file}'")
        except Exception as e:
            print(f"Error saving metrics CSV: {e}")
    else:
        print("Metrics DataFrame is empty.")


    if trajectory:
        trajectory_df = save_trajectory_to_csv(trajectory, trajectory_csv_file)
    else:
        trajectory_df = pd.DataFrame() # Empty DataFrame

    if not trajectory_df.empty:
        plot_trajectory_3d(trajectory_df, filename=plot_3d_file)
        plot_trajectory_2d(trajectory_df, filename=plot_2d_file)

        # --- Print Summary ---
        print("\n--- Summary of Registration Results ---")
        # Calculate averages only on valid results (where RMSE >= 0)
        valid_metrics = metrics_df[metrics_df['final_rmse'] >= 0]
        if not valid_metrics.empty:
            avg_fitness = valid_metrics['final_fitness'].mean()
            avg_rmse = valid_metrics['final_rmse'].mean()
            print(f"Average final ICP fitness (valid pairs): {avg_fitness:.4f}")
            print(f"Average final ICP RMSE (valid pairs): {avg_rmse:.4f}")
        else:
            print("No valid registration metrics found to calculate averages.")


        # Calculate total distance from valid trajectory points
        valid_traj_points = trajectory_df[['x', 'y', 'z']].dropna().values
        if len(valid_traj_points) > 1:
            diffs = np.diff(valid_traj_points, axis=0)
            segment_lengths = np.sqrt(np.sum(diffs**2, axis=1))
            total_distance = np.sum(segment_lengths)
            print(f"Total estimated trajectory distance: {total_distance:.4f} meters")

            # Save summary report
            try:
                with open(summary_report_file, "w") as f:
                    f.write("Point Cloud Registration Summary (RANSAC + Point-to-Plane ICP)\n")
                    f.write("============================================================\n\n")
                    f.write(f"Number of point clouds processed: {len(pcd_files)}\n")
                    f.write(f"Parameters Used:\n")
                    f.write(f"- RANSAC/ICP Voxel Size: {ransac_voxel_size}\n")
                    f.write(f"- ICP Distance Threshold: {icp_threshold}\n")
                    f.write(f"- ICP Max Iterations: {icp_max_iteration}\n")
                    f.write(f"- KNN for Normals: {knn_normals}\n\n")
                    f.write(f"Registration Results (Averages on valid pairs):\n")
                    if not valid_metrics.empty:
                        f.write(f"- Average Final ICP Fitness: {avg_fitness:.4f}\n")
                        f.write(f"- Average Final ICP RMSE: {avg_rmse:.4f}\n\n")
                        # Include average RANSAC stats if desired
                        avg_ransac_f = valid_metrics['ransac_fitness'][valid_metrics['ransac_fitness'] >= 0].mean()
                        avg_ransac_e = valid_metrics['ransac_rmse'][valid_metrics['ransac_rmse'] >= 0].mean()
                        f.write(f"- Average RANSAC Fitness (Initial Guess Eval): {avg_ransac_f:.4f}\n")
                        f.write(f"- Average RANSAC RMSE (Initial Guess Eval): {avg_ransac_e:.4f}\n\n")
                    else:
                         f.write("- No valid registration results to average.\n\n")


                    f.write(f"Trajectory Information:\n")
                    f.write(f"- Total distance traveled: {total_distance:.4f} meters\n")
                    f.write(f"- Bounding box (X): {trajectory_df['x'].min():.2f} to {trajectory_df['x'].max():.2f} meters\n")
                    f.write(f"- Bounding box (Y): {trajectory_df['y'].min():.2f} to {trajectory_df['y'].max():.2f} meters\n")
                    f.write(f"- Bounding box (Z): {trajectory_df['z'].min():.2f} to {trajectory_df['z'].max():.2f} meters\n")
                print(f"Saved summary report to '{summary_report_file}'")
            except Exception as e:
                print(f"Error writing summary report: {e}")
        else:
            print("Not enough valid trajectory points to calculate distance.")

    else:
        print("Trajectory data is empty, cannot plot or calculate distance.")


    # --- Visualize Final Map ---
    visualize_combined_point_cloud(registered_pcds, trajectory,
                                   downsample_voxel_size=map_vis_voxel_size,
                                   save_path=global_map_file)

    print("\n--- Script execution finished ---")


if __name__ == "__main__":
    # This block executes the main function when the script is run directly.
    main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Found 100 point cloud files.

Starting registration process using RANSAC + Point-to-Plane ICP...
Parameters: RANSAC/ICP Voxel=0.05, ICP Thresh=0.05, ICP Iter=100, Normals KNN=30
Preprocessing cloud 0 (pointcloud_0000.pcd)...
----------------------------------------
Processing pair 0-1 : (pointcloud_0000.pcd -> pointcloud_0004.pcd)
----------------------------------------
Preprocessing cloud 1...
Running RANSAC for initial alignment...
RANSAC Initial Guess - Fitness: 0.9648, RMSE: 0.0186
Running Point-to-Plane ICP for refinement...
Registration 1/99 - ICP Fitness: 0.964623, ICP RMSE: 0.017889
----------------------------------------
Processing pair 1-2 : (pointcloud_0004.pcd -> pointcloud_0008.pcd)
----------------------------------------
Preprocessing cloud 2...
Running RANSAC for initial alignment...
RANSAC Initial Gue

2

In [2]:
# -*- coding: utf-8 -*-
"""
Modified experimental script incorporating robust RANSAC initialization.
Compares Random Orthogonal vs. RANSAC (feature-based) initialization
for Point-to-Point ICP.
"""

import numpy as np
import open3d as o3d
import copy
import os
from scipy.stats import ortho_group
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate # Used in analyze_results

# === Core Helper Functions (Partially from 2022006_5_complete_ransac_p2plane.py) ===

def load_point_cloud(file_path):
    """Load a point cloud from file."""
    try:
        pcd = o3d.io.read_point_cloud(file_path)
        if not pcd.has_points():
            print(f"Warning: No points found in {file_path}")
            return None
        return pcd
    except Exception as e:
        print(f"Error loading point cloud {file_path}: {e}")
        return None

def preprocess_point_cloud(pcd, voxel_size=0.05, estimate_normals=True, knn=30):
    """Preprocess point cloud: downsample and estimate normals. (Robust version)"""
    if pcd is None:
        return None

    # Create a copy to avoid modifying the original
    processed_pcd = copy.deepcopy(pcd)

    # Downsample using voxel grid filter
    # print(f"    Downsampling with voxel size {voxel_size}...") # Verbose
    processed_pcd = processed_pcd.voxel_down_sample(voxel_size)
    if not processed_pcd.has_points():
        print("Warning: Point cloud has no points after downsampling.")
        return None

    # Estimate normals if requested
    if estimate_normals:
        if not processed_pcd.has_normals(): # Only estimate if not already present
             # print(f"    Estimating normals with KNN={knn}...") # Verbose
             search_param = o3d.geometry.KDTreeSearchParamKNN(knn=knn)
             processed_pcd.estimate_normals(search_param=search_param)
             # Orient normals consistently
             processed_pcd.orient_normals_consistent_tangent_plane(k=knn)
             if not processed_pcd.has_normals():
                  print("Warning: Failed to estimate normals.")

    return processed_pcd

def prepare_point_clouds(source, target, voxel_size=0.05, estimate_normals=False, knn=30):
    """Prepare point clouds for registration using the robust preprocess function"""
    if source is None or target is None:
        return None, None
    source_processed = preprocess_point_cloud(source, voxel_size, estimate_normals, knn)
    target_processed = preprocess_point_cloud(target, voxel_size, estimate_normals, knn)
    return source_processed, target_processed

def get_random_orthogonal_matrix():
    """Generate a random valid 4x4 transformation matrix."""
    R = ortho_group.rvs(3)
    t = np.random.rand(3, 1) * 0.1 # Small random translation (consistent with first script)

    initial_transform = np.eye(4)
    initial_transform[:3, :3] = R
    initial_transform[:3, 3] = t.squeeze()

    return initial_transform

# --- RANSAC Function (from 2022006_5_complete_ransac_p2plane.py) ---
def get_initial_transform_from_ransac(source, target, voxel_size, with_normals=True):
    """Use RANSAC for initial alignment (global registration).
       This version relies on FPFH features and requires normals.
    """
    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Invalid input point clouds for RANSAC.")
        return np.eye(4)

    distance_threshold = voxel_size * 1.5 # Max correspondence distance for RANSAC

    if not with_normals:
         print("Error: This RANSAC implementation requires normals for FPFH feature matching.")
         return np.eye(4) # Return Identity if normals are required but not requested/available

    # --- Feature Matching RANSAC (Requires Normals) ---
    if not source.has_normals() or not target.has_normals():
        print("Error: Normals required for feature matching RANSAC but not found.")
        # Attempt to estimate them if missing? Or just fail? Let's fail for clarity.
        # Consider adding estimation here if needed:
        # source = preprocess_point_cloud(source, voxel_size=voxel_size, estimate_normals=True)
        # target = preprocess_point_cloud(target, voxel_size=voxel_size, estimate_normals=True)
        # if not source.has_normals() or not target.has_normals():
        #    print("Error: Failed to estimate normals for RANSAC.")
        #    return np.eye(4)
        return np.eye(4) # Return Identity if normals are required but missing

    # print("    Computing FPFH features...") # Verbose
    source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))
    target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100))

    # Corrected Check for FPFH data validity
    source_data_valid = source_fpfh is not None and hasattr(source_fpfh, 'data') and source_fpfh.data.shape[0] > 0 and source_fpfh.data.shape[1] > 0
    target_data_valid = target_fpfh is not None and hasattr(target_fpfh, 'data') and target_fpfh.data.shape[0] > 0 and target_fpfh.data.shape[1] > 0

    if not source_data_valid or not target_data_valid:
         print("Error: FPFH feature computation failed or resulted in empty features.")
         # print(f"Source FPFH shape: {source_fpfh.data.shape if source_data_valid else 'Invalid/None'}") # Debug
         # print(f"Target FPFH shape: {target_fpfh.data.shape if target_data_valid else 'Invalid/None'}") # Debug
         return np.eye(4)

    # print("    Running RANSAC based on features...") # Verbose
    # Check Open3D version for mutual_filter compatibility if needed
    try:
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source, target, source_fpfh, target_fpfh,
            mutual_filter=True, # Requires Open3D >= 0.13
            max_correspondence_distance=distance_threshold,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # Point-to-Point for RANSAC transform estimation itself
            ransac_n=4, # Typically 3 or 4 points
            checkers=[
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            # criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999) # Max iterations, confidence
            criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500) # Alternative higher params from first script
        )
    except TypeError: # Handle potential absence of mutual_filter in older Open3D
         print("    Running RANSAC without mutual_filter (older Open3D version?).")
         result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source, target, source_fpfh, target_fpfh,
            # mutual_filter=True, # Removed
            max_correspondence_distance=distance_threshold,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            ransac_n=4,
            checkers=[
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
        )
    except Exception as e:
         print(f"An unexpected error occurred during RANSAC: {e}")
         return np.eye(4)

    # Check if RANSAC result is valid (sometimes returns identity on failure)
    if np.allclose(result.transformation, np.identity(4)):
        # Evaluate to see if identity is actually a good fit or a failure
        eval_ransac_fail = o3d.pipelines.registration.evaluate_registration(
            source, target, distance_threshold, result.transformation)
        if eval_ransac_fail.fitness < 0.1: # Arbitrary threshold for failure
             print("Warning: RANSAC returned identity matrix, likely due to insufficient correspondences or failure.")
        else:
             print("Info: RANSAC returned identity matrix, but it seems to be a reasonable fit.")


    return result.transformation
# --- End of RANSAC Function ---


def point_to_point_icp(source, target, initial_transform=None,
                        max_iteration=30, threshold=0.02,
                        estimate_normals_in_icp=False, # Separate flag for ICP's normal needs
                        knn=30, voxel_size_icp=0.05): # Voxel size for potential internal preprocessing
    """Run Point-to-Point ICP with various configurations"""
    if source is None or target is None:
        print("Error: Invalid input point clouds for ICP.")
        return { # Return structure indicating failure
            'transformation': np.eye(4), 'initial_fitness': 0.0, 'initial_rmse': -1.0,
            'final_fitness': 0.0, 'final_rmse': -1.0, 'fitness_improvement': 0.0,
            'rmse_improvement': 0.0, 'transform_error': -1.0 }

    # Use already processed clouds if available, otherwise process here
    # NOTE: The experimental setup prepares clouds beforehand. This is more for standalone use.
    # Let's assume source and target are the *original* clouds passed in run_experiments
    # And we might downsample/prepare them *again* here if needed for ICP specifically.
    # For this experimental script, let's rely on run_experiments preparing them.
    # We still need copies to avoid modifying originals passed to this function.
    source_processed = copy.deepcopy(source)
    target_processed = copy.deepcopy(target)

    # Downsample *if* desired specifically for ICP stage (optional)
    # source_processed = source_processed.voxel_down_sample(voxel_size_icp)
    # target_processed = target_processed.voxel_down_sample(voxel_size_icp)

    # Estimate normals *if* required by a specific ICP variant (not P2P here)
    # or if needed for evaluation metrics perhaps. Point-to-Point doesn't strictly need them.
    if estimate_normals_in_icp:
         print("Warning: Estimating normals within P2P ICP function, usually not needed.")
         if not source_processed.has_normals():
              source_processed.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn))
              source_processed.orient_normals_consistent_tangent_plane(k=knn)
         if not target_processed.has_normals():
              target_processed.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn))
              target_processed.orient_normals_consistent_tangent_plane(k=knn)

    # Default initial transform if not provided (use random orthogonal)
    if initial_transform is None:
        print("ICP Info: No initial transform provided, using random orthogonal matrix.")
        initial_transform = get_random_orthogonal_matrix()

    # Evaluate initial alignment
    # Use try-except as evaluate_registration can fail with empty clouds etc.
    initial_fitness = 0.0
    initial_rmse = -1.0 # Use -1 to indicate potential failure/invalidity
    try:
        if source_processed.has_points() and target_processed.has_points():
             evaluation = o3d.pipelines.registration.evaluate_registration(
                 source_processed, target_processed, threshold, initial_transform)
             initial_fitness = evaluation.fitness
             initial_rmse = evaluation.inlier_rmse
        else:
             print("Warning: Cannot evaluate initial alignment due to empty processed clouds.")
    except Exception as e:
        print(f"Warning: Error during initial evaluation: {e}")


    # Run Point-to-Point ICP
    final_fitness = 0.0
    final_rmse = -1.0
    transformation = initial_transform # Default to initial if ICP fails badly
    try:
        if source_processed.has_points() and target_processed.has_points():
            reg_p2p = o3d.pipelines.registration.registration_icp(
                source_processed, target_processed, threshold, initial_transform,
                o3d.pipelines.registration.TransformationEstimationPointToPoint(), # Explicitly P2P
                o3d.pipelines.registration.ICPConvergenceCriteria(
                    relative_fitness=1e-6, relative_rmse=1e-6, # Add relative criteria
                    max_iteration=max_iteration)
                )
            transformation = reg_p2p.transformation
            final_fitness = reg_p2p.fitness
            final_rmse = reg_p2p.inlier_rmse
        else:
            print("Warning: Skipping ICP run due to empty processed clouds.")
            transformation = initial_transform # Keep initial guess if no points

    except Exception as e:
         print(f"Error during ICP registration: {e}")
         # Keep initial_transform as the result in case of error
         transformation = initial_transform
         # Metrics remain at failure values


    # Calculate transformation error (Frobenius norm of difference from initial)
    # This isn't super meaningful without ground truth, but kept for consistency with original script
    transform_error = np.linalg.norm(transformation - initial_transform)

    # Calculate improvements, handle invalid initial RMSE
    fitness_improvement = final_fitness - initial_fitness
    rmse_improvement = (initial_rmse - final_rmse) if initial_rmse >= 0 and final_rmse >= 0 else 0.0 # Avoid negative initial RMSE issues

    return {
        'transformation': transformation,
        'initial_fitness': initial_fitness,
        'initial_rmse': initial_rmse,
        'final_fitness': final_fitness,
        'final_rmse': final_rmse,
        'fitness_improvement': fitness_improvement,
        'rmse_improvement': rmse_improvement,
        'transform_error': transform_error # Kept, but interpretation needs care
    }


def run_experiments(source_file, target_file):
    """Run multiple experiments comparing Random vs RANSAC initialization for P2P ICP."""
    # Load point clouds (original resolution)
    source_orig = load_point_cloud(source_file)
    target_orig = load_point_cloud(target_file)
    if source_orig is None or target_orig is None:
        print("Error loading source or target file, cannot run experiments.")
        return pd.DataFrame() # Return empty DataFrame

    # Define hyperparameter combinations to test for ICP
    # Use thresholds relevant to the expected noise/voxel size
    thresholds = [0.02, 0.05, 0.1, 0.2] # Adjusted from original experimental script
    max_iterations = [50, 100, 200]   # ICP iterations
    ransac_voxel_size = 0.05          # Voxel size for RANSAC preprocessing
    knn_normals = 30                  # KNN for normal estimation

    # Initialize results storage
    results = []

    # --- Experiment Group 1: Random Orthogonal Initialization ---
    print("-" * 30)
    print("Running experiments with RANDOM ORTHOGONAL matrix initialization...")
    print("-" * 30)
    for threshold in thresholds:
        for max_iter in max_iterations:
            print(f"Random Init - Threshold: {threshold}, Max Iter: {max_iter}")
            # Generate a new random initial transform for each run
            initial_transform_rand = get_random_orthogonal_matrix()

            # Run Point-to-Point ICP using the original point clouds
            # ICP function handles internal processing/copying
            result = point_to_point_icp(
               source_orig, target_orig, initial_transform_rand,
               max_iter, threshold,
               estimate_normals_in_icp=False, # P2P doesn't need normals
               knn=knn_normals,
               voxel_size_icp=ransac_voxel_size # Pass voxel size if ICP does internal downsampling (currently commented out inside)
            )

            # Store results
            results.append({
                'initialization': 'Random Orthogonal',
                'threshold': threshold,
                'max_iterations': max_iter,
                'ransac_voxel_size': None, # Not applicable
                'knn_normals': knn_normals,
                **result # Unpack dict returned by point_to_point_icp
            })

    # --- Experiment Group 2: RANSAC (Feature-Based) Initialization ---
    print("\n" + "-" * 30)
    print("Running experiments with RANSAC (Feature-Based) initialization...")
    print("-" * 30)

    # Prepare point clouds *once* for RANSAC (downsampled WITH normals)
    print(f"Preparing clouds for RANSAC (voxel={ransac_voxel_size}, knn={knn_normals})...")
    source_for_ransac, target_for_ransac = prepare_point_clouds(
        source_orig, target_orig, ransac_voxel_size, estimate_normals=True, knn=knn_normals)

    ransac_initial_transform = np.eye(4) # Default to identity if RANSAC fails
    ransac_succeeded = False
    if source_for_ransac is not None and target_for_ransac is not None:
        try:
            print("Running RANSAC to get initial transform...")
            ransac_initial_transform = get_initial_transform_from_ransac(
                source_for_ransac, target_for_ransac, ransac_voxel_size, with_normals=True) # Must use with_normals=True

            # Basic check if RANSAC likely succeeded (not identity and reasonable fitness)
            if not np.allclose(ransac_initial_transform, np.identity(4)):
                 eval_ransac = o3d.pipelines.registration.evaluate_registration(
                      source_for_ransac, target_for_ransac, ransac_voxel_size * 1.5, ransac_initial_transform)
                 print(f"RANSAC Initial Guess Eval - Fitness: {eval_ransac.fitness:.4f}, RMSE: {eval_ransac.inlier_rmse:.4f}")
                 if eval_ransac.fitness > 0.05: # Threshold for considering RANSAC successful
                      ransac_succeeded = True
                 else:
                      print("Warning: RANSAC fitness low, may proceed with identity matrix.")
                      ransac_initial_transform = np.eye(4) # Fallback to identity if fitness is too low
            else:
                 # If it returned identity, check fitness (handled inside get_initial_transform_from_ransac now)
                 eval_ransac = o3d.pipelines.registration.evaluate_registration(
                      source_for_ransac, target_for_ransac, ransac_voxel_size * 1.5, ransac_initial_transform)
                 print(f"RANSAC returned identity matrix. Eval - Fitness: {eval_ransac.fitness:.4f}, RMSE: {eval_ransac.inlier_rmse:.4f}")
                 # Decide if identity is acceptable or counts as failure
                 if eval_ransac.fitness > 0.1: # If identity is actually a good fit
                      ransac_succeeded = True
                 # else: RANSAC essentially failed, keep identity as fallback

        except Exception as e:
            print(f"Error during RANSAC execution: {e}. Using identity matrix as initial transform.")
            ransac_initial_transform = np.eye(4) # Fallback on error
    else:
        print("Error preparing clouds for RANSAC. Using identity matrix as initial transform.")
        ransac_initial_transform = np.eye(4) # Fallback if preparation failed


    if not ransac_succeeded:
        print("Warning: RANSAC initialization failed or produced poor result. ICP using identity/fallback transform.")

    # Run ICP experiments using the *single* RANSAC result as initial transform
    for threshold in thresholds:
        for max_iter in max_iterations:
            print(f"RANSAC Init - Threshold: {threshold}, Max Iter: {max_iter}")

            # Run Point-to-Point ICP using the original clouds and the RANSAC transform
            result = point_to_point_icp(
                source_orig, target_orig, ransac_initial_transform, # Use the computed RANSAC transform
                max_iter, threshold,
                estimate_normals_in_icp=False, # P2P doesn't need normals
                knn=knn_normals,
                voxel_size_icp=ransac_voxel_size
            )

            # Store results
            results.append({
                'initialization': 'RANSAC (Features)', # More specific name
                'threshold': threshold,
                'max_iterations': max_iter,
                'ransac_voxel_size': ransac_voxel_size,
                'knn_normals': knn_normals,
                **result
            })


    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(results)

    return results_df

def analyze_results(results_df):
    """Analyze experimental results and find best hyperparameters"""
    if results_df.empty:
        print("Result DataFrame is empty. Cannot analyze.")
        return pd.DataFrame()

    print("\n--- EXPERIMENTAL RESULTS SUMMARY ---")

    # Handle cases where metrics might be invalid (e.g., -1.0 for RMSE)
    results_df_valid = results_df[results_df['final_rmse'] >= 0].copy() # Work with valid results

    if results_df_valid.empty:
        print("No valid results found (all final_rmse < 0). Cannot perform analysis.")
        # Print raw results maybe?
        print("\nRaw Results:")
        print(tabulate(results_df, headers='keys', tablefmt='pretty'))
        return results_df # Return original empty/invalid df

    # Find best configurations based on valid results
    best_rmse_row = results_df_valid.loc[results_df_valid['final_rmse'].idxmin()]
    best_fitness_row = results_df_valid.loc[results_df_valid['final_fitness'].idxmax()]
    best_rmse_improvement_row = results_df_valid.loc[results_df_valid['rmse_improvement'].idxmax()]
    best_fitness_improvement_row = results_df_valid.loc[results_df_valid['fitness_improvement'].idxmax()]

    # Grouped comparisons using valid results
    init_method_comparison = results_df_valid.groupby('initialization').agg({
        'initial_rmse': 'mean', 'final_rmse': 'mean', 'rmse_improvement': 'mean',
        'initial_fitness': 'mean', 'final_fitness': 'mean', 'fitness_improvement': 'mean',
        'final_rmse': 'count' # Add count to see sample size
    }).rename(columns={'final_rmse': 'mean_final_rmse', 'count': 'num_valid_runs'})

    threshold_comparison = results_df_valid.groupby('threshold').agg({
        'initial_rmse': 'mean', 'final_rmse': 'mean', 'rmse_improvement': 'mean',
        'initial_fitness': 'mean', 'final_fitness': 'mean', 'fitness_improvement': 'mean',
         'final_rmse': 'count'
    }).rename(columns={'final_rmse': 'mean_final_rmse', 'count': 'num_valid_runs'})

    iterations_comparison = results_df_valid.groupby('max_iterations').agg({
        'initial_rmse': 'mean', 'final_rmse': 'mean', 'rmse_improvement': 'mean',
        'initial_fitness': 'mean', 'final_fitness': 'mean', 'fitness_improvement': 'mean',
         'final_rmse': 'count'
    }).rename(columns={'final_rmse': 'mean_final_rmse', 'count': 'num_valid_runs'})


    # --- Print Summary ---
    print("\nBest configuration by final RMSE:")
    print(best_rmse_row[['initialization', 'threshold', 'max_iterations',
                         'initial_rmse', 'final_rmse', 'rmse_improvement']].to_string())
    print("\nTransformation matrix for best RMSE configuration:")
    # Ensure transformation is numpy array for printing
    best_transform_np = np.array(best_rmse_row['transformation'])
    print(best_transform_np)

    print("\nBest configuration by final fitness:")
    print(best_fitness_row[['initialization', 'threshold', 'max_iterations',
                            'initial_fitness', 'final_fitness', 'fitness_improvement']].to_string())

    print("\nComparison by initialization method (based on valid runs):")
    print(tabulate(init_method_comparison, headers='keys', tablefmt='pretty'))

    print("\nComparison by threshold value (based on valid runs):")
    print(tabulate(threshold_comparison, headers='keys', tablefmt='pretty'))

    print("\nComparison by max iterations (based on valid runs):")
    print(tabulate(iterations_comparison, headers='keys', tablefmt='pretty'))

    # Prepare summary table for saving (include all results, mark invalid ones if needed)
    summary_table = results_df[['initialization', 'threshold', 'max_iterations',
                               'ransac_voxel_size', 'knn_normals',
                               'initial_rmse', 'final_rmse', 'rmse_improvement',
                               'initial_fitness', 'final_fitness', 'fitness_improvement']]

    return summary_table

def visualize_best_result(source, target, best_transform, window_name="Best Alignment Result"):
    """Visualize point clouds with the best transformation"""
    if source is None or target is None:
        print("Cannot visualize, source or target is None.")
        return
    if not source.has_points() or not target.has_points():
         print("Cannot visualize, source or target has no points.")
         return

    # Transform source to target using best transformation
    source_transformed = copy.deepcopy(source)
    source_transformed.transform(best_transform)

    # Visualize
    source_temp = copy.deepcopy(source)
    source_temp.paint_uniform_color([1, 0, 0])  # Red for source

    target_temp = copy.deepcopy(target)
    target_temp.paint_uniform_color([0, 1, 0])  # Green for target

    source_transformed_temp = copy.deepcopy(source_transformed)
    source_transformed_temp.paint_uniform_color([0, 0, 1])  # Blue for transformed source

    o3d.visualization.draw_geometries([source_temp, target_temp, source_transformed_temp],
                                      window_name=window_name)

# === Main Execution Block ===

def main():
    # --- Configuration ---
    # IMPORTANT: Update this path to your dataset location!
    dataset_dir = "C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds"
    output_dir = "." # Directory to save outputs (CSV)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    summary_csv_file = os.path.join(output_dir, "p2p_icp_ransac_vs_random_experiments.csv")
    # --- End Configuration ---

    # Get sorted list of point cloud files
    try:
        all_files = os.listdir(dataset_dir)
    except FileNotFoundError:
        print(f"Error: Dataset directory not found: {dataset_dir}")
        print("Please update the 'dataset_dir' variable in the script.")
        return

    pcd_files = sorted([f for f in all_files if f.startswith("pointcloud_") and f.endswith(".pcd")])

    if len(pcd_files) < 2:
        print(f"Error: Need at least 2 PCD files in '{dataset_dir}' for comparison.")
        return

    # Select two consecutive point clouds for experiments
    # (e.g., using clouds 10 and 11 - adjust indices if needed/desired)
    source_idx = 10
    target_idx = 11
    if max(source_idx, target_idx) >= len(pcd_files):
        print(f"Error: Indices {source_idx}, {target_idx} are out of range for the {len(pcd_files)} available files.")
        # Fallback to first two files
        source_idx, target_idx = 0, 1
        print(f"Using files {pcd_files[source_idx]} and {pcd_files[target_idx]} instead.")

    source_file = os.path.join(dataset_dir, pcd_files[source_idx])
    target_file = os.path.join(dataset_dir, pcd_files[target_idx])

    print(f"Running Point-to-Point ICP experiments comparing initializations on:")
    print(f"  Source: {pcd_files[source_idx]}")
    print(f"  Target: {pcd_files[target_idx]}")

    # --- Run Experiments ---
    results_df = run_experiments(source_file, target_file)

    if results_df.empty:
         print("\nExperiment execution failed or yielded no results.")
         return

    # --- Analyze Results ---
    summary_table = analyze_results(results_df) # analysis also prints summary to console

    # --- Save Summary Table ---
    if not summary_table.empty:
        try:
            summary_table.to_csv(summary_csv_file, index=False)
            print(f"\nFull experimental results saved to '{summary_csv_file}'")
        except Exception as e:
            print(f"\nError saving summary CSV: {e}")

    # --- Visualize Best Result ---
    if not results_df.empty and 'final_rmse' in results_df.columns:
         results_df_valid = results_df[results_df['final_rmse'] >= 0]
         if not results_df_valid.empty:
              best_rmse_idx = results_df_valid['final_rmse'].idxmin()
              best_transform = results_df_valid.loc[best_rmse_idx, 'transformation']
              best_config_info = results_df_valid.loc[best_rmse_idx, ['initialization', 'threshold', 'max_iterations']]

              print(f"\nVisualizing best result (lowest valid final_rmse):")
              print(best_config_info.to_string())

              # Load original clouds again for visualization
              source_orig_vis = load_point_cloud(source_file)
              target_orig_vis = load_point_cloud(target_file)
              visualize_best_result(source_orig_vis, target_orig_vis, best_transform,
                                    window_name=f"Best Result: {best_config_info['initialization']} T={best_config_info['threshold']} iter={best_config_info['max_iterations']}")
         else:
              print("\nCannot visualize best result: No valid runs found in the results.")
    else:
        print("\nCannot visualize best result: Results DataFrame is empty or missing 'final_rmse'.")


    print("\n--- Script execution finished ---")


if __name__ == "__main__":
    main()

Running Point-to-Point ICP experiments comparing initializations on:
  Source: pointcloud_0040.pcd
  Target: pointcloud_0044.pcd
------------------------------
Running experiments with RANDOM ORTHOGONAL matrix initialization...
------------------------------
Random Init - Threshold: 0.02, Max Iter: 50
Random Init - Threshold: 0.02, Max Iter: 100
Random Init - Threshold: 0.02, Max Iter: 200
Random Init - Threshold: 0.05, Max Iter: 50
Random Init - Threshold: 0.05, Max Iter: 100
Random Init - Threshold: 0.05, Max Iter: 200
Random Init - Threshold: 0.1, Max Iter: 50
Random Init - Threshold: 0.1, Max Iter: 100
Random Init - Threshold: 0.1, Max Iter: 200
Random Init - Threshold: 0.2, Max Iter: 50
Random Init - Threshold: 0.2, Max Iter: 100
Random Init - Threshold: 0.2, Max Iter: 200

------------------------------
Running experiments with RANSAC (Feature-Based) initialization...
------------------------------
Preparing clouds for RANSAC (voxel=0.05, knn=30)...
Running RANSAC to get initial 

## 3

In [3]:
# -*- coding: utf-8 -*-
"""
Script to visualize and analyze a specific, pre-determined
registration transformation result between two point clouds.
Uses robust methods for analysis where applicable.
"""

import numpy as np
import open3d as o3d
import copy
import os

# === Helper Functions (Adapted for Analysis Context) ===

def load_point_cloud(file_path):
    """Load a point cloud from file."""
    try:
        pcd = o3d.io.read_point_cloud(file_path)
        if not pcd.has_points():
            print(f"Warning: No points found in {file_path}")
            return None
        return pcd
    except Exception as e:
        print(f"Error loading point cloud {file_path}: {e}")
        return None

def visualize_registration_result(source, target, transformation, eval_threshold=0.05):
    """
    Visualize the source, target, and transformed source point clouds.
    Also evaluates the registration using the provided threshold.
    """
    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Cannot visualize registration. Source or target cloud is invalid.")
        return None

    # Create a copy of the source point cloud and transform it
    source_transformed = copy.deepcopy(source)
    source_transformed.transform(transformation)

    # Color the point clouds
    source_temp = copy.deepcopy(source)
    source_temp.paint_uniform_color([1, 0, 0])  # Red for original source

    target_temp = copy.deepcopy(target)
    target_temp.paint_uniform_color([0, 1, 0])  # Green for target

    source_transformed_temp = copy.deepcopy(source_transformed)
    source_transformed_temp.paint_uniform_color([0, 0, 1])  # Blue for transformed source

    # Create a coordinate frame to visualize the transformation's effect relative to origin
    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.5, origin=[0, 0, 0]) # Represents global origin

    # Visualize all three + origin frame
    print("\n--- Visualization 1: Source (Red), Target (Green), Transformed Source (Blue) ---")
    print("Coordinate frame shows the global reference frame")
    try:
        o3d.visualization.draw_geometries(
            [source_temp, target_temp, source_transformed_temp, coord_frame],
            window_name="Registration: Source(R), Target(G), Transformed(B)")
    except Exception as e:
        print(f"Error during visualization 1: {e}")


    # Visualize just the target and transformed source to see alignment quality
    print("\n--- Visualization 2: Target (Green) and Transformed Source (Blue) ---")
    try:
        o3d.visualization.draw_geometries(
            [target_temp, source_transformed_temp, coord_frame],
            window_name="Registration: Target(G), Transformed(B)")
    except Exception as e:
        print(f"Error during visualization 2: {e}")


    # Calculate metrics to quantify the quality of alignment
    print(f"\n--- Alignment Metrics (using threshold = {eval_threshold}) ---")
    # Use the specified threshold for evaluation
    try:
        evaluation = o3d.pipelines.registration.evaluate_registration(
            source_transformed, target, eval_threshold) # Use the specified threshold

        print(f"Fitness: {evaluation.fitness:.6f} (proportion of inliers)")
        print(f"Inlier RMSE: {evaluation.inlier_rmse:.6f} (Root Mean Square Error of inlier correspondences)")
    except Exception as e:
        print(f"Error during registration evaluation: {e}")
        evaluation = None # Indicate failure

    return source_transformed # Return the transformed cloud

def analyze_transformation(transformation):
    """
    Analyze the 4x4 transformation matrix and extract key components.
    Uses robust Euler angle calculation with gimbal lock handling.
    """
    print("\n--- Transformation Matrix Analysis ---")
    if transformation is None or transformation.shape != (4, 4):
        print("Error: Invalid transformation matrix provided.")
        return None

    # Extract rotation matrix
    rotation = transformation[:3, :3]

    # Extract translation vector
    translation = transformation[:3, 3]

    # Check if rotation matrix is orthogonal (within tolerance)
    is_orthogonal = np.allclose(np.dot(rotation, rotation.T), np.eye(3), atol=1e-6) # Use absolute tolerance

    # Calculate determinant to check if it's a proper rotation (det ≈ 1)
    determinant = np.linalg.det(rotation)
    is_proper_rotation = np.isclose(determinant, 1.0)

    # Calculate the Euler angles using robust method (ZYX convention: Yaw, Pitch, Roll)
    roll_deg, pitch_deg, yaw_deg = None, None, None
    euler_angles_valid = False
    try:
        # Check for valid rotation matrix before attempting angles
        if is_orthogonal and is_proper_rotation:
            # Calculate pitch (around Y axis)
            pitch = np.arcsin(-rotation[2, 0])

            # Check for gimbal lock (pitch near +/- 90 degrees)
            if np.isclose(np.cos(pitch), 0.0):
                # Gimbal lock: Set roll to 0 and calculate yaw differently
                print("Note: Potential gimbal lock detected (pitch near +/- 90 deg).")
                roll = 0.0
                yaw = np.arctan2(rotation[0, 1], rotation[1, 1]) # Yaw based on remaining DOFs
            else:
                # Normal case: Calculate roll (around X axis) and yaw (around Z axis)
                roll = np.arctan2(rotation[2, 1], rotation[2, 2])
                yaw = np.arctan2(rotation[1, 0], rotation[0, 0])

            # Convert to degrees
            roll_deg, pitch_deg, yaw_deg = np.degrees(roll), np.degrees(pitch), np.degrees(yaw)
            euler_angles_valid = True
        elif not is_orthogonal:
            print("Warning: Rotation matrix is not orthogonal. Euler angles not calculated.")
        elif not is_proper_rotation:
            print(f"Warning: Rotation matrix determinant is {determinant:.4f} (not 1.0). May include reflection/scaling. Euler angles not calculated.")

    except ValueError as e:
         print(f"Error calculating Euler angles: {e}. Matrix may be ill-conditioned.")
    except Exception as e:
         print(f"An unexpected error occurred during Euler angle calculation: {e}")


    # Calculate the magnitude of translation
    translation_magnitude = np.linalg.norm(translation)

    # Print the analysis
    print(f"\nTranslation Vector (X, Y, Z): [{translation[0]:.4f}, {translation[1]:.4f}, {translation[2]:.4f}]")
    print(f"Translation Magnitude: {translation_magnitude:.4f} meters (assuming input units are meters)")

    print(f"\nRotation Matrix:\n{rotation}")
    print(f"Is Orthogonal (R * R^T ≈ I): {is_orthogonal}")
    print(f"Determinant: {determinant:.6f} (Proper rotation if ≈ 1)")

    if euler_angles_valid:
        print(f"\nEuler Angles (Roll, Pitch, Yaw) [deg]: ({roll_deg:.2f}, {pitch_deg:.2f}, {yaw_deg:.2f})")
        print("  (Convention: Applied as Z(yaw) -> Y(pitch) -> X(roll))")
    else:
        print("\nEuler Angles: Not calculated due to issues with the rotation matrix.")

    return {
        'translation': translation,
        'translation_magnitude': translation_magnitude,
        'rotation_matrix': rotation,
        'is_orthogonal': is_orthogonal,
        'determinant': determinant,
        'is_proper_rotation': is_proper_rotation,
        'angles_deg': (roll_deg, pitch_deg, yaw_deg) if euler_angles_valid else None
    }

# === Main Execution Block ===

def main():
    # --- Configuration ---
    # IMPORTANT: Update this path to your dataset location!
    dataset_dir = "C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds"
    output_dir = "." # Directory to save transformed cloud

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Indices of the point clouds used in the experiment that produced the 'best_transform'
    # (Consistent with the previous experimental script example)
    source_idx = 10
    target_idx = 11

    # --- IMPORTANT ---
    # Define the best transformation matrix found from your previous experiments.
    # Replace the values below with the actual 4x4 matrix corresponding to the
    # best result (e.g., lowest final_rmse) from the experimental script's output
    # (e.g., from 'p2p_icp_ransac_vs_random_experiments.csv' or console log).
    # The values here are placeholders based on previous user input.
    best_transform = np.array([
        [-0.46210878, -0.87531359, -0.14241349, 0.01718528],
        [ 0.71034064, -0.46148236,  0.53146044, 0.47404969],
        [ 0.53091586, -0.14443044, -0.83502586, 0.08875592],
        [ 0.0,          0.0,          0.0,         1.0       ]
    ])
    # --- END IMPORTANT ---

    transformed_pcd_save_path = os.path.join(output_dir, f"transformed_source_{source_idx}_to_{target_idx}.pcd")
    # --- End Configuration ---


    # Get sorted list of point cloud files
    try:
        all_files = os.listdir(dataset_dir)
    except FileNotFoundError:
        print(f"Error: Dataset directory not found: {dataset_dir}")
        print("Please update the 'dataset_dir' variable in the script.")
        return

    pcd_files = sorted([f for f in all_files if f.startswith("pointcloud_") and f.endswith(".pcd")])

    if len(pcd_files) < max(source_idx, target_idx) + 1:
        print(f"Error: Not enough PCD files found in '{dataset_dir}' for the selected indices ({source_idx}, {target_idx}).")
        print(f"Found {len(pcd_files)} files, need at least {max(source_idx, target_idx) + 1}.")
        return

    source_file = os.path.join(dataset_dir, pcd_files[source_idx])
    target_file = os.path.join(dataset_dir, pcd_files[target_idx])

    print(f"Analyzing transformation for:")
    print(f"  Source: {pcd_files[source_idx]} ({source_file})")
    print(f"  Target: {pcd_files[target_idx]} ({target_file})")
    print(f"Using pre-defined 'best_transform':\n{best_transform}")

    # Load point clouds
    source = load_point_cloud(source_file)
    target = load_point_cloud(target_file)

    # Check if loading was successful
    if source is None or target is None:
        print("\nError loading one or both point clouds. Aborting.")
        return

    print(f"\nSource point cloud loaded with {len(source.points)} points.")
    print(f"Target point cloud loaded with {len(target.points)} points.")


    # Visualize the registration result using the hardcoded best transform
    transformed_source = visualize_registration_result(source, target, best_transform, eval_threshold=0.05)


    # Analyze the transformation matrix
    analysis_results = analyze_transformation(best_transform)


    # Save the transformed point cloud if visualization and analysis were successful
    if transformed_source is not None and analysis_results is not None:
        try:
            o3d.io.write_point_cloud(transformed_pcd_save_path, transformed_source)
            print(f"\nTransformed source point cloud saved to '{transformed_pcd_save_path}'")
        except Exception as e:
            print(f"\nError saving transformed point cloud: {e}")


    # Provide qualitative conclusion (remains the same as user provided)
    print("\n--- Qualitative Conclusion ---")
    print("""
    Reasons for the results (general observations):
    1. The quality of registration depends heavily on the overlap between point clouds and scene geometry.
    2. Feature-based methods (like RANSAC with FPFH) or methods using normals (like Point-to-Plane ICP) often improve alignment by incorporating more geometric information.
    3. Appropriate parameter tuning (voxel size, thresholds, iterations) is crucial for good performance.
    4. The transformation matrix represents the estimated motion (translation and rotation) of the sensor (e.g., TurtleBot) between the capture times of the source and target frames.
    5. Alignment metrics (Fitness, RMSE) quantify the geometric consistency after applying the transformation. High fitness and low RMSE generally indicate better alignment.
    6. Near-perfect alignment (RMSE ≈ 0) on real data might indicate overfitting, especially if noise is present, or simply very high overlap and good algorithm performance.
    """)

    print("\n--- Script execution finished ---")


if __name__ == "__main__":
    main()

Analyzing transformation for:
  Source: pointcloud_0040.pcd (C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds\pointcloud_0040.pcd)
  Target: pointcloud_0044.pcd (C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds\pointcloud_0044.pcd)
Using pre-defined 'best_transform':
[[-0.46210878 -0.87531359 -0.14241349  0.01718528]
 [ 0.71034064 -0.46148236  0.53146044  0.47404969]
 [ 0.53091586 -0.14443044 -0.83502586  0.08875592]
 [ 0.          0.          0.          1.        ]]

Source point cloud loaded with 22712 points.
Target point cloud loaded with 22675 points.

--- Visualization 1: Source (Red), Target (Green), Transformed Source (Blue) ---
Coordinate frame shows the global reference frame

--- Visualization 2: Target (Green) and Transformed Source (Blue) ---

--- Alignment Metrics (using threshold = 0.05) ---
Fitness: 0.010215 (proportion of inliers)
Inlier RMSE: 0.034181 (Root Mean Square Error of inlier correspondences)

--- Transformation Matrix Analy

## 4.

In [4]:
# -*- coding: utf-8 -*-
"""
Script for Part 4: Sequential Point Cloud Registration using RANSAC + Point-to-Point ICP.

This script processes a sequence of point clouds to estimate the sensor trajectory
and generate a combined global point cloud map. It uses RANSAC for robust initial
alignment and **Point-to-Point ICP** for refinement, as specified in Part 4 of the assignment.
"""

import numpy as np
import open3d as o3d
import copy
import os
from scipy.stats import ortho_group # Only needed as a potential fallback, not primary method
import pandas as pd
import matplotlib.pyplot as plt
# Import necessary for 3D plotting
from mpl_toolkits.mplot3d import Axes3D

# === Core Helper Functions (Reused and verified) ===

def load_point_cloud(file_path):
    """Load a point cloud from file."""
    try:
        pcd = o3d.io.read_point_cloud(file_path)
        if not pcd.has_points():
            print(f"Warning: No points found in {file_path}")
            return None
        # Remove non-finite points proactively
        pcd_clean = pcd.remove_non_finite_points(remove_nan=True, remove_infinite=True)
        if not pcd_clean.has_points():
             print(f"Warning: No finite points remaining in {file_path} after cleaning.")
             return None
        # if len(pcd_clean.points) < len(pcd.points):
        #     print(f"Info: Removed {len(pcd.points) - len(pcd_clean.points)} non-finite points from {file_path}.")
        return pcd_clean
    except Exception as e:
        print(f"Error loading point cloud {file_path}: {e}")
        return None

def visualize_point_clouds(source, target, source_transformed=None, window_name="Point Cloud Alignment"):
    """Visualize original source (red), target (green), and optionally transformed source (blue)."""
    vis_list = []
    source_temp, target_temp, source_transformed_temp = None, None, None # Initialize

    if source is not None and source.has_points():
        source_temp = copy.deepcopy(source)
        source_temp.paint_uniform_color([1, 0, 0])  # Red for source
        vis_list.append(source_temp)

    if target is not None and target.has_points():
        target_temp = copy.deepcopy(target)
        target_temp.paint_uniform_color([0, 1, 0])  # Green for target
        vis_list.append(target_temp)

    if source_transformed is not None and source_transformed.has_points():
        source_transformed_temp = copy.deepcopy(source_transformed)
        source_transformed_temp.paint_uniform_color([0, 0, 1])  # Blue for transformed source
        vis_list.append(source_transformed_temp)

    if vis_list:
        print(f"Visualizing: Source ({len(source_temp.points) if source_temp else 0}), Target ({len(target_temp.points) if target_temp else 0}), Transformed ({len(source_transformed_temp.points) if source_transformed_temp else 0}) points.")
        o3d.visualization.draw_geometries(vis_list, window_name=window_name)
    else:
        print("Warning: Nothing to visualize.")


def preprocess_point_cloud(pcd, voxel_size=0.05, estimate_normals=True, knn=30):
    """
    Preprocess point cloud: downsample, estimate normals (if requested), and orient normals.
    Normals are needed for RANSAC FPFH even if ICP is Point-to-Point.
    """
    if pcd is None:
        print("Warning: preprocess_point_cloud received None input.")
        return None

    # Create a copy to avoid modifying the original
    processed_pcd = copy.deepcopy(pcd)

    if not processed_pcd.has_points():
         print("Warning: Input pcd to preprocess has no points.")
         return None # Return None if input is empty

    # Voxel Downsampling
    # print(f"    Input points before downsampling: {len(processed_pcd.points)}") # Debug
    processed_pcd = processed_pcd.voxel_down_sample(voxel_size)
    # print(f"    Points after downsampling (voxel={voxel_size}): {len(processed_pcd.points)}") # Debug
    if not processed_pcd.has_points():
        print("Warning: Point cloud has no points after downsampling.")
        return None

    # Estimate Normals (Required for FPFH-based RANSAC)
    if estimate_normals:
        if not processed_pcd.has_normals():
             # print(f"    Estimating normals with KNN={knn}...") # Verbose
             try:
                 processed_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=knn))
             except RuntimeError as e:
                 print(f"Warning: Normal estimation failed: {e}. Cloud might be too sparse or planar.")
                 # Continue without normals, RANSAC will likely fail later.
                 return processed_pcd

             if not processed_pcd.has_normals():
                 print("Warning: Failed to estimate normals even after attempting.")
             else:
                 # print("    Orienting normals...") # Verbose
                 # Orient normals consistently
                 try:
                     processed_pcd.orient_normals_consistent_tangent_plane(k=knn) # Use same k as KNN
                 except RuntimeError as e:
                     print(f"Warning: Normal orientation failed: {e}. Using unoriented normals.")

    return processed_pcd

# --- RANSAC Function (Feature-based - Requires Normals) ---
def get_initial_transform_from_ransac(source, target, voxel_size):
    """
    Use RANSAC based on FPFH features for initial alignment.
    Requires source and target point clouds to have normals estimated.
    """
    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Invalid input point clouds for RANSAC.")
        return np.eye(4), 0.0, -1.0 # Return identity and invalid metrics

    distance_threshold = voxel_size * 1.5 # Max correspondence distance for RANSAC

    # Check for normals, which are essential for FPFH
    if not source.has_normals() or not target.has_normals():
        print("Error: Normals required for feature matching RANSAC but not found.")
        return np.eye(4), 0.0, -1.0

    # print("    Computing FPFH features...") # Verbose
    try:
        # Use KDTreeSearchParamHybrid for FPFH feature computation
        radius_normal = voxel_size * 2
        radius_feature = voxel_size * 5
        # print(f"    FPFH params: radius_normal={radius_normal}, radius_feature={radius_feature}") # Debug
        source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source,
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
        target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target,
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    except Exception as e:
        print(f"Error computing FPFH features: {e}")
        return np.eye(4), 0.0, -1.0

    # Check for FPFH data validity
    source_data_valid = source_fpfh is not None and hasattr(source_fpfh, 'data') and source_fpfh.data.shape[0] == len(source.points)
    target_data_valid = target_fpfh is not None and hasattr(target_fpfh, 'data') and target_fpfh.data.shape[0] == len(target.points)

    if not source_data_valid or not target_data_valid:
         print(f"Error: FPFH feature computation failed or resulted in incorrect number of features.")
         print(f"Source points: {len(source.points)}, Source FPFH: {source_fpfh.data.shape if source_data_valid else 'Invalid'}")
         print(f"Target points: {len(target.points)}, Target FPFH: {target_fpfh.data.shape if target_data_valid else 'Invalid'}")
         return np.eye(4), 0.0, -1.0

    # print("    Running RANSAC based on features...") # Verbose
    result = None
    ransac_fitness = 0.0
    ransac_rmse = -1.0
    ransac_transform = np.eye(4)
    # Set RANSAC parameters
    ransac_criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(
        max_iteration=4000000, # High number of iterations
        confidence=0.999)      # Confidence level

    # Define checkers
    checkers = [
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ]

    # RANSAC based on feature matching
    try:
        # Attempt with mutual_filter=True (available in newer Open3D versions)
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source, target, source_fpfh, target_fpfh,
            mutual_filter=True, # Enable mutual filter
            max_correspondence_distance=distance_threshold,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # P2P for internal RANSAC estimation
            ransac_n=3, # Minimum 3 points for Point-to-Point estimation
            checkers=checkers,
            criteria=ransac_criteria
        )
    except TypeError:
        # Fallback for older Open3D versions without mutual_filter argument
        print("    Running RANSAC without mutual_filter (older Open3D version?).")
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source, target, source_fpfh, target_fpfh,
            # mutual_filter=True, # Argument removed
            max_correspondence_distance=distance_threshold,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            ransac_n=3,
            checkers=checkers,
            criteria=ransac_criteria
        )
    except Exception as e:
         print(f"An unexpected error occurred during RANSAC feature matching: {e}")
         return np.eye(4), 0.0, -1.0 # Return identity on error

    if result is not None:
        ransac_transform = result.transformation
        ransac_fitness = result.fitness
        ransac_rmse = result.inlier_rmse
        # Check if RANSAC result is identity matrix, which might indicate failure
        if np.allclose(ransac_transform, np.identity(4), atol=1e-6):
             # Evaluate identity transformation to see if it's actually a failure
             eval_identity = o3d.pipelines.registration.evaluate_registration(
                 source, target, distance_threshold, np.identity(4))
             if eval_identity.fitness < 0.1: # Low fitness suggests identity is a poor match -> RANSAC likely failed
                  print("Warning: RANSAC returned identity matrix with low fitness, likely failed.")
             else: # High fitness suggests clouds might already be aligned or overlap is small/ambiguous
                  print("Info: RANSAC returned identity matrix, but initial fitness is reasonable.")
    else:
        print("Warning: RANSAC registration function returned None.")
        # Keep transform as identity, metrics indicate failure

    return ransac_transform, ransac_fitness, ransac_rmse


# --- ICP Function (Modified for Point-to-Point Only) ---
def run_point_to_point_icp(source, target, initial_transform,
                           max_iteration=100, threshold=0.05):
    """
    Run Point-to-Point ICP refinement.
    Assumes source and target are already preprocessed (e.g., downsampled).
    Does NOT require normals for the algorithm itself.
    """
    if source is None or target is None or not source.has_points() or not target.has_points():
        print("Error: Invalid input point clouds for Point-to-Point ICP.")
        return np.eye(4), 0.0, -1.0, 0.0, -1.0 # Return identity and invalid metrics

    if initial_transform is None:
        print("Warning: initial_transform is None in run_point_to_point_icp. Using identity.")
        initial_transform = np.eye(4)

    # Evaluate initial alignment (using the provided initial_transform)
    initial_fitness = 0.0
    initial_rmse = -1.0
    try:
        evaluation = o3d.pipelines.registration.evaluate_registration(
            source, target, threshold, initial_transform)
        initial_fitness = evaluation.fitness
        initial_rmse = evaluation.inlier_rmse
        # print(f"    P2P ICP Initial Alignment - Fitness: {initial_fitness:.4f}, RMSE: {initial_rmse:.4f}") # Verbose
    except Exception as e:
        print(f"Warning: Could not evaluate initial alignment for P2P ICP: {e}")

    # Run Point-to-Point ICP
    transformation = initial_transform # Default to initial if ICP fails
    final_fitness = 0.0
    final_rmse = -1.0
    try:
        reg_result = o3d.pipelines.registration.registration_icp(
            source, target, threshold, initial_transform,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(), # Explicitly Point-to-Point
            o3d.pipelines.registration.ICPConvergenceCriteria(
                relative_fitness=1e-7, relative_rmse=1e-7, max_iteration=max_iteration) # Convergence criteria
            )

        # Get results
        transformation = reg_result.transformation
        final_fitness = reg_result.fitness
        final_rmse = reg_result.inlier_rmse
    except Exception as e:
        print(f"Error during Point-to-Point ICP execution: {e}")
        # Keep initial transform and failure metrics

    return transformation, initial_fitness, initial_rmse, final_fitness, final_rmse


# === Sequence Registration Function (RANSAC + Point-to-Point ICP) ===

def register_all_point_clouds_p2p(dataset_dir, pcd_files,
                                  preprocess_voxel_size=0.05,
                                  icp_threshold=0.05,
                                  icp_max_iteration=100,
                                  knn=30):
    """
    Register all point clouds sequentially using RANSAC initial alignment + Point-to-Point ICP refinement.
    Args:
        dataset_dir (str): Path to the directory containing PCD files.
        pcd_files (list): Sorted list of PCD filenames.
        preprocess_voxel_size (float): Voxel size for downsampling AND RANSAC normal estimation radius.
        icp_threshold (float): Correspondence distance threshold for Point-to-Point ICP.
        icp_max_iteration (int): Maximum iterations for Point-to-Point ICP.
        knn (int): Number of neighbors for normal estimation (needed for RANSAC).

    Returns:
        tuple: (list of globally transformed original clouds, list of global poses, pandas.DataFrame of metrics)
               Returns (None, None, None) on critical failure.
    """
    if not pcd_files:
        print("Error: No PCD files provided.")
        return None, None, None

    trajectory = []             # Stores the global transformation matrix for each frame
    registered_pcds_global = [] # Stores original resolution clouds transformed to global frame
    registration_metrics = []   # Stores metrics for each registration step

    # --- Initialize with the first point cloud ---
    first_pcd_path = os.path.join(dataset_dir, pcd_files[0])
    print(f"Loading initial cloud 0: {pcd_files[0]}...")
    first_pcd_orig = load_point_cloud(first_pcd_path)
    if first_pcd_orig is None:
        print("Error: Could not load the first point cloud. Aborting.")
        return None, None, None

    # Preprocess the first cloud: Downsample AND Estimate Normals (for RANSAC)
    print(f"Preprocessing cloud 0 (voxel={preprocess_voxel_size}, knn={knn})...")
    first_pcd_processed = preprocess_point_cloud(first_pcd_orig,
                                                 voxel_size=preprocess_voxel_size,
                                                 estimate_normals=True, # Normals needed for RANSAC FPFH
                                                 knn=knn)
    if first_pcd_processed is None or not first_pcd_processed.has_points():
        print("Error: Failed to preprocess the first point cloud. Aborting.")
        return None, None, None
    # Check if normals were estimated successfully for the first cloud (needed as target later)
    if not first_pcd_processed.has_normals():
        print(f"Error: Failed to estimate normals for the first point cloud {pcd_files[0]}, needed for RANSAC. Aborting.")
        return None, None, None

    # The first cloud defines the origin of the global coordinate system
    global_transform = np.eye(4)
    trajectory.append(copy.deepcopy(global_transform))
    registered_pcds_global.append(copy.deepcopy(first_pcd_orig)) # Store original resolution cloud at origin

    # Keep track of the *processed* previous cloud for registration
    prev_pcd_processed = first_pcd_processed
    # Keep track of the last successful *relative* transform as a fallback for failed pairs
    last_successful_relative_transform = np.eye(4)

    # --- Process subsequent pairs ---
    num_clouds = len(pcd_files)
    for i in range(1, num_clouds):
        print("-" * 50)
        print(f"Processing Pair {i-1}-{i}: [{pcd_files[i-1]} -> {pcd_files[i]}] ({i}/{num_clouds-1})")
        print("-" * 50)

        # Initialize metrics dictionary for this pair
        current_metrics = {
            'pair_index': f"{i-1}-{i}", 'source_file': pcd_files[i], 'target_file': pcd_files[i-1],
            'ransac_init_fitness': -1.0, 'ransac_init_rmse': -1.0,
            'icp_initial_fitness': -1.0, 'icp_initial_rmse': -1.0,
            'icp_final_fitness': -1.0, 'icp_final_rmse': -1.0,
            'status': 'Failed - Load/Preprocess' # Default status
        }
        relative_transform_prev_to_curr = np.eye(4) # Default relative transform is identity

        # Load current point cloud (original resolution)
        current_pcd_path = os.path.join(dataset_dir, pcd_files[i])
        current_pcd_orig = load_point_cloud(current_pcd_path)
        if current_pcd_orig is None:
            print(f"Warning: Skipping pair {i-1}-{i} due to loading error for {pcd_files[i]}.")
            print("         Estimating pose using last successful relative motion.")
            relative_transform_prev_to_curr = last_successful_relative_transform # Use fallback motion
            current_metrics['status'] = 'Failed - Load Error'
        else:
            # Preprocess current point cloud: Downsample AND Estimate Normals (for RANSAC)
            print(f"Preprocessing cloud {i} (voxel={preprocess_voxel_size}, knn={knn})...")
            current_pcd_processed = preprocess_point_cloud(current_pcd_orig,
                                                          voxel_size=preprocess_voxel_size,
                                                          estimate_normals=True, # Normals needed for RANSAC
                                                          knn=knn)

            if current_pcd_processed is None or not current_pcd_processed.has_points():
                 print(f"Warning: Skipping pair {i-1}-{i} due to preprocessing error (no points) for {pcd_files[i]}.")
                 print("         Estimating pose using last successful relative motion.")
                 relative_transform_prev_to_curr = last_successful_relative_transform
                 current_metrics['status'] = 'Failed - Preprocessing Error (No Points)'
            # Check specifically if normals were estimated, as RANSAC requires them
            elif not current_pcd_processed.has_normals():
                 print(f"Warning: Skipping pair {i-1}-{i} due to failed normal estimation for {pcd_files[i]} (required by RANSAC).")
                 print("         Estimating pose using last successful relative motion.")
                 relative_transform_prev_to_curr = last_successful_relative_transform
                 current_metrics['status'] = 'Failed - Preprocessing Error (No Normals)'
            else:
                # --- RANSAC for Initial Alignment (using processed clouds with normals) ---
                print("Running RANSAC (Feature-based) for initial alignment...")
                # RANSAC estimates transform from source (current) to target (prev)
                # T_curr_to_prev_ransac
                initial_transform_ransac, ransac_f, ransac_e = get_initial_transform_from_ransac(
                    current_pcd_processed, prev_pcd_processed, # Use processed clouds WITH NORMALS
                    preprocess_voxel_size)
                current_metrics['ransac_init_fitness'] = ransac_f
                current_metrics['ransac_init_rmse'] = ransac_e
                print(f"RANSAC Initial Guess Eval - Fitness: {ransac_f:.4f}, RMSE: {ransac_e:.4f}")

                ransac_failed = False
                # Consider RANSAC failed if fitness is very low or RMSE is invalid
                if ransac_f < 0.05 or ransac_e < 0: # Stricter fitness check for RANSAC usefulness
                    print("Warning: RANSAC initial alignment failed or result is poor. ICP will start from identity.")
                    ransac_failed = True
                    initial_transform_for_icp = np.eye(4) # Fallback: Start ICP from identity
                else:
                    initial_transform_for_icp = initial_transform_ransac # Use RANSAC result

                # --- Point-to-Point ICP Refinement ---
                print("Running Point-to-Point ICP for refinement...")
                # ICP refines the transform from source (current) to target (prev)
                # Input: current_pcd_processed, prev_pcd_processed, T_curr_to_prev_initial
                # Output: T_curr_to_prev_refined
                transformation_icp, init_f, init_e, final_fitness, final_rmse = run_point_to_point_icp(
                    current_pcd_processed, prev_pcd_processed, # Use processed clouds (normals not used by P2P ICP)
                    initial_transform=initial_transform_for_icp, # Start from RANSAC result (or identity fallback)
                    max_iteration=icp_max_iteration,
                    threshold=icp_threshold
                )
                current_metrics['icp_initial_fitness'] = init_f
                current_metrics['icp_initial_rmse'] = init_e
                current_metrics['icp_final_fitness'] = final_fitness
                current_metrics['icp_final_rmse'] = final_rmse
                print(f"P2P ICP Refinement Result - Fitness: {final_fitness:.6f}, RMSE: {final_rmse:.6f}")

                # --- Determine Relative Motion and Handle Failures ---
                icp_succeeded = True
                # Define failure conditions for Point-to-Point ICP (can be less strict on RMSE than P2Plane)
                # Check fitness, valid RMSE. RMSE check threshold might need tuning for P2P.
                if final_fitness < 0.1 or final_rmse < 0 or final_rmse > icp_threshold * 10: # Relaxed RMSE check for P2P
                     print(f"Warning: Point-to-Point ICP result might be poor for pair {i-1}-{i} (Fitness: {final_fitness:.3f}, RMSE: {final_rmse:.3f}).")
                     icp_succeeded = False

                if icp_succeeded:
                    # ICP success: Calculate relative transform T_prev_to_curr = inv(T_curr_to_prev_refined)
                    try:
                        relative_transform_prev_to_curr = np.linalg.inv(transformation_icp)
                        last_successful_relative_transform = relative_transform_prev_to_curr # Update fallback
                        current_metrics['status'] = 'Success'
                    except np.linalg.LinAlgError:
                         print("Error: Failed to invert ICP transformation matrix. Using last successful motion.")
                         relative_transform_prev_to_curr = last_successful_relative_transform
                         current_metrics['status'] = 'Fallback - Matrix Inversion Error'

                else:
                    # ICP failed: Decide fallback strategy
                    if not ransac_failed and ransac_f > 0.1: # If RANSAC seemed okay-ish, use its result cautiously
                        print("         Using RANSAC transform directly due to poor P2P ICP result.")
                        try:
                            relative_transform_prev_to_curr = np.linalg.inv(initial_transform_ransac)
                            current_metrics['status'] = 'Fallback - RANSAC Only'
                        except np.linalg.LinAlgError:
                            print("Error: Failed to invert RANSAC transformation matrix. Using last successful motion.")
                            relative_transform_prev_to_curr = last_successful_relative_transform
                            current_metrics['status'] = 'Fallback - Matrix Inversion Error'
                    else: # Both RANSAC and ICP seem poor, use last known good motion
                        print("         Both RANSAC and P2P ICP poor. Using last successful relative transform.")
                        relative_transform_prev_to_curr = last_successful_relative_transform
                        current_metrics['status'] = 'Fallback - Last Good Motion'

                # Update the previous *processed* cloud for the next iteration's target
                prev_pcd_processed = current_pcd_processed

        # --- Update Global Transformation and Store Results ---
        # Current global pose T_global_curr = T_global_prev * T_prev_to_curr
        # Add safeguard against non-invertible relative transforms before multiplying
        if np.linalg.det(relative_transform_prev_to_curr) != 0:
             global_transform = global_transform @ relative_transform_prev_to_curr
        else:
             print(f"Warning: Skipping global transform update for frame {i} due to singular relative transform. Pose will be same as previous.")
             # Keep global_transform unchanged

        trajectory.append(copy.deepcopy(global_transform))

        # Transform the *original* high-resolution current point cloud to the global frame
        if current_pcd_orig is not None:
            transformed_pcd_orig = copy.deepcopy(current_pcd_orig)
            transformed_pcd_orig.transform(global_transform)
            registered_pcds_global.append(transformed_pcd_orig)
        else:
            registered_pcds_global.append(o3d.geometry.PointCloud()) # Append empty placeholder

        # Store metrics for this pair
        registration_metrics.append(current_metrics)

        # --- End of loop for one pair ---

    print("\nSequential registration process finished.")
    # Convert metrics list to DataFrame
    metrics_df = pd.DataFrame(registration_metrics)
    return registered_pcds_global, trajectory, metrics_df

# === Visualization and Analysis Functions (Unchanged, verified plotting calls) ===

def save_trajectory_to_csv(trajectory, output_path):
    """Save the trajectory (poses) to a CSV file including Euler angles."""
    positions = []
    if not trajectory:
        print("Warning: Trajectory is empty. Cannot save to CSV.")
        return pd.DataFrame()

    print(f"Saving trajectory with {len(trajectory)} poses...")
    for frame_idx, transform in enumerate(trajectory):
        if transform is None or not np.all(np.isfinite(transform)):
             print(f"Warning: Skipping frame {frame_idx} due to invalid transform matrix.")
             positions.append({
                'frame': frame_idx, 'x': np.nan, 'y': np.nan, 'z': np.nan,
                'roll_deg': np.nan, 'pitch_deg': np.nan, 'yaw_deg': np.nan
             })
             continue

        x, y, z = transform[:3, 3]
        rotation = transform[:3, :3]

        # ZYX Euler angles (yaw, pitch, roll) - Handle potential gimbal lock issues
        roll_deg, pitch_deg, yaw_deg = np.nan, np.nan, np.nan # Default to NaN
        try:
            # Check for valid rotation matrix properties before angle calculation
            is_orthogonal = np.allclose(np.dot(rotation, rotation.T), np.eye(3), atol=1e-5)
            determinant = np.linalg.det(rotation)
            is_proper_rotation = np.isclose(determinant, 1.0, atol=1e-5)

            if is_orthogonal and is_proper_rotation:
                # Check for pitch singularity (gimbal lock)
                sin_pitch = -rotation[2, 0]
                if np.abs(sin_pitch) > 0.99999: # Close to +/- 1
                    # Gimbal lock: pitch is +/- 90. Roll is effectively 0 relative to this frame.
                    # print(f"Info: Gimbal lock detected at frame {frame_idx}. Setting roll=0.") # Optional info
                    pitch_rad = np.arcsin(sin_pitch) # +/- pi/2
                    roll_rad = 0.0
                    yaw_rad = np.arctan2(rotation[0, 1], rotation[1, 1]) # Use atan2(R12, R22) or other stable combination
                else:
                    # Normal case
                    pitch_rad = np.arcsin(sin_pitch)
                    roll_rad = np.arctan2(rotation[2, 1], rotation[2, 2]) # atan2(R32, R33)
                    yaw_rad = np.arctan2(rotation[1, 0], rotation[0, 0])  # atan2(R21, R11)

                roll_deg, pitch_deg, yaw_deg = np.degrees(roll_rad), np.degrees(pitch_rad), np.degrees(yaw_rad)
            else:
                 print(f"Warning: Invalid rotation matrix at frame {frame_idx} (Orthogonal: {is_orthogonal}, Det: {determinant:.4f}). Skipping angle calculation.")

        except (ValueError, RuntimeError, np.linalg.LinAlgError) as e: # Catch math errors or other issues
             print(f"Error calculating Euler angles for frame {frame_idx}: {e}. Setting to NaN.")

        positions.append({
            'frame': frame_idx,
            'x': x, 'y': y, 'z': z,
            'roll_deg': roll_deg, 'pitch_deg': pitch_deg, 'yaw_deg': yaw_deg
        })

    trajectory_df = pd.DataFrame(positions)
    try:
        trajectory_df.to_csv(output_path, index=False) # Don't write pandas index
        print(f"Trajectory saved to '{output_path}'")
    except Exception as e:
        print(f"Error saving trajectory to CSV: {e}")
        return pd.DataFrame() # Return empty on save error

    return trajectory_df

def plot_trajectory_3d(trajectory_df, filename='turtlebot_trajectory_3d.png', title_suffix=""):
    """Plot the estimated 3D trajectory."""
    if trajectory_df is None or trajectory_df.empty or trajectory_df[['x', 'y', 'z']].isnull().all().all():
        print("Cannot plot empty or invalid trajectory.")
        return None, None

    # Drop rows with NaN positions for plotting
    plot_df = trajectory_df.dropna(subset=['x', 'y', 'z'])
    if plot_df.empty:
        print("No valid (non-NaN) trajectory points to plot.")
        return None, None

    print(f"Plotting 3D trajectory with {len(plot_df)} points...")
    fig = plt.figure(figsize=(10, 8))
    # Ensure using Axes3D for subplot
    ax = fig.add_subplot(111, projection='3d')

    x = plot_df['x'].values
    y = plot_df['y'].values
    z = plot_df['z'].values

    # Plot path
    ax.plot(x, y, z, 'b-', linewidth=2, label='Robot Path')
    # Scatter waypoints, color-coded by time (index)
    scatter = ax.scatter(x, y, z, c=plot_df.index, cmap='viridis', s=20, label='Waypoints (time)', alpha=0.7, depthshade=True)
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], c='lime', s=100, label='Start', edgecolors='k', depthshade=True)
    ax.scatter(x[-1], y[-1], z[-1], c='red', s=100, label='End', edgecolors='k', depthshade=True)

    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(f'Estimated TurtleBot 3D Trajectory{title_suffix}')

    # Add colorbar for time
    cbar = fig.colorbar(scatter, ax=ax, shrink=0.6)
    cbar.set_label('Frame Index (Time)')

    # Make axes equal scale for better visualization
    try:
        x_min, x_max = x.min(), x.max()
        y_min, y_max = y.min(), y.max()
        z_min, z_max = z.min(), z.max()
        max_range = np.array([x_max-x_min, y_max-y_min, z_max-z_min]).max()
        if max_range == 0: max_range = 1.0 # Avoid division by zero
        mid_x = (x_max+x_min) * 0.5
        mid_y = (y_max+y_min) * 0.5
        mid_z = (z_max+z_min) * 0.5

        ax.set_xlim(mid_x - max_range/2.0, mid_x + max_range/2.0)
        ax.set_ylim(mid_y - max_range/2.0, mid_y + max_range/2.0)
        ax.set_zlim(mid_z - max_range/2.0, mid_z + max_range/2.0)
        # Attempt to set aspect ratio - might vary slightly by matplotlib version
        ax.set_aspect('equal', adjustable='box')
        print("Set 3D plot axes to equal scale.")
    except Exception as e:
        print(f"Warning: Could not set equal axes for 3D plot: {e}")

    ax.legend()
    ax.grid(True)
    try:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"3D trajectory plot saved to '{filename}'")
    except Exception as e:
        print(f"Error saving 3D plot: {e}")
    # plt.show() # Optional: Show plot interactively
    plt.close(fig) # Close plot to free memory
    return fig, ax

def plot_trajectory_2d(trajectory_df, filename='turtlebot_trajectory_2d.png', title_suffix=""):
    """Plot the estimated 2D (X-Y) trajectory projection."""
    if trajectory_df is None or trajectory_df.empty or trajectory_df[['x', 'y']].isnull().all().all():
        print("Cannot plot empty or invalid 2D trajectory.")
        return

    # Drop rows with NaN positions for plotting
    plot_df = trajectory_df.dropna(subset=['x', 'y'])
    if plot_df.empty:
        print("No valid (non-NaN) 2D trajectory points to plot.")
        return

    print(f"Plotting 2D trajectory with {len(plot_df)} points...")
    plt.figure(figsize=(10, 10))

    x = plot_df['x'].values
    y = plot_df['y'].values

    plt.plot(x, y, 'b-', linewidth=2, label='Robot Path')
    scatter = plt.scatter(x, y, c=plot_df.index, cmap='viridis', s=20, label='Waypoints (time)', alpha=0.7)
    plt.scatter(x[0], y[0], c='lime', s=100, label='Start', edgecolors='k')
    plt.scatter(x[-1], y[-1], c='red', s=100, label='End', edgecolors='k')

    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.title(f'Estimated TurtleBot 2D Trajectory (Top-Down View){title_suffix}')
    plt.grid(True)
    plt.axis('equal') # Ensure aspect ratio is equal
    plt.legend()
    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Frame Index (Time)')

    try:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"2D trajectory plot saved to '{filename}'")
    except Exception as e:
        print(f"Error saving 2D plot: {e}")
    # plt.show() # Optional: Show plot interactively
    plt.close() # Close plot


def visualize_and_save_combined_point_cloud(registered_pcds_global, trajectory=None,
                                            downsample_voxel_size=0.05,
                                            save_path="global_registered_map.pcd",
                                            window_name="Combined Global Map"):
    """
    Visualize the combined registered point cloud map and optionally the trajectory.
    Saves the combined map (downsampled for manageability).
    """
    if not registered_pcds_global:
        print("No registered point clouds to visualize or save.")
        return

    map_vis_elements = []
    combined_pcd_for_saving = o3d.geometry.PointCloud()
    valid_cloud_count = 0

    # Combine clouds (downsample for saving/visualization)
    print(f"Combining {len(registered_pcds_global)} clouds for map (using voxel size {downsample_voxel_size})...")
    num_clouds = len(registered_pcds_global)
    for i, pcd_orig_global in enumerate(registered_pcds_global):
        if pcd_orig_global is None or not pcd_orig_global.has_points():
            # print(f"Skipping empty cloud index {i} for map generation.") # Verbose
            continue

        # Downsample the globally transformed cloud
        pcd_down_global = pcd_orig_global.voxel_down_sample(voxel_size=downsample_voxel_size)
        if not pcd_down_global.has_points():
            continue # Skip if downsampling removed all points

        # --- For Visualization ---
        pcd_colored = copy.deepcopy(pcd_down_global)
        # Assign distinct color using HSV space for visualization
        hue = i / float(num_clouds) if num_clouds > 0 else 0
        color = plt.cm.hsv(hue)[:3] # Get RGB from HSV colormap
        pcd_colored.paint_uniform_color(color)
        map_vis_elements.append(pcd_colored)

        # --- For Saving ---
        # Add the downsampled version to the combined cloud for saving
        combined_pcd_for_saving += pcd_down_global
        valid_cloud_count += 1

    print(f"Combined {valid_cloud_count} valid point clouds into the map.")

    # Add trajectory visualization if provided
    if trajectory is not None and len(trajectory) > 1:
        print("Adding trajectory visualization elements...")
        # Filter out invalid trajectory points before creating lineset
        valid_points = []
        valid_indices_map = {} # Map original index to valid index in valid_points
        original_indices = [] # Keep track of original indices of valid points
        for idx, T in enumerate(trajectory):
            if T is not None and np.all(np.isfinite(T)):
                 valid_indices_map[idx] = len(valid_points)
                 valid_points.append(T[:3, 3])
                 original_indices.append(idx)

        if len(valid_points) > 1:
            points_np = np.array(valid_points)
            lines = []
            # Create lines connecting consecutive *valid* points based on original sequence
            for k in range(len(original_indices) - 1):
                idx1_orig = original_indices[k]
                idx2_orig = original_indices[k+1]
                # Connect if they were originally consecutive or just connect the valid sequence?
                # Let's connect consecutive points in the valid_points list
                lines.append([k, k + 1]) # Connect index k to k+1 in the valid_points array

            if lines: # Ensure there are lines to draw
                line_set = o3d.geometry.LineSet()
                line_set.points = o3d.utility.Vector3dVector(points_np)
                line_set.lines = o3d.utility.Vector2iVector(lines)
                # Color trajectory line (e.g., bright red)
                line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in range(len(lines))])
                map_vis_elements.append(line_set)

            # Add coordinate frames at key poses (e.g., every Nth valid pose)
            step = max(1, len(valid_points) // 15) # Show ~15 frames along the valid trajectory
            print(f"Adding coordinate frames at step {step}...")
            for i in range(0, len(valid_points), step):
                frame_idx_orig = original_indices[i] # Get original index
                transform = trajectory[frame_idx_orig]
                # Double check validity before creating frame
                if transform is not None and np.all(np.isfinite(transform)):
                    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0]) # Smaller frames
                    coord_frame.transform(transform)
                    map_vis_elements.append(coord_frame)
        else:
            print("Warning: Not enough valid trajectory points (>1) to draw lines.")


    # Visualize combined map and trajectory
    if map_vis_elements:
        print("Visualizing combined map and trajectory (ESC key to close)...")
        try:
            o3d.visualization.draw_geometries(map_vis_elements, window_name=window_name)
        except Exception as e:
            print(f"Error during visualization: {e}")
    else:
        print("No elements (clouds or trajectory) to visualize.")

    # Save the combined point cloud (downsampled version)
    if combined_pcd_for_saving.has_points():
        print(f"Saving combined map with {len(combined_pcd_for_saving.points)} points...")
        try:
            # Ensure normals are recalculated or removed if inconsistent after merging
            # combined_pcd_for_saving.estimate_normals() # Optional: Estimate normals for the whole map
            o3d.io.write_point_cloud(save_path, combined_pcd_for_saving, write_ascii=False) # Save as binary for efficiency
            print(f"Saved global registered point cloud map to '{save_path}'")
        except Exception as e:
            print(f"Error saving combined point cloud: {e}")
    else:
        print("No points in the combined cloud to save.")


def create_summary_report(metrics_df, trajectory_df, params, output_file):
    """Creates a text summary report of the registration process."""
    print(f"Generating summary report to '{output_file}'...")
    method_name = "RANSAC + Point-to-Point ICP" # Update method name
    try:
        with open(output_file, "w") as f:
            f.write(f"Point Cloud Registration Summary ({method_name})\n")
            f.write("=" * (31 + len(method_name)) + "\n\n") # Adjust underline
            f.write(f"Dataset Directory: {params['dataset_dir']}\n")
            f.write(f"Number of point clouds processed: {params['num_files']}\n\n")

            f.write(f"Parameters Used:\n")
            f.write(f"- Preprocessing Voxel Size: {params['preprocess_voxel_size']}\n")
            f.write(f"- ICP Distance Threshold: {params['icp_threshold']}\n")
            f.write(f"- ICP Max Iterations: {params['icp_max_iteration']}\n")
            f.write(f"- KNN for Normals (RANSAC): {params['knn']}\n\n")

            f.write(f"Registration Results (Pairwise):\n")
            # Ensure status column exists before filtering
            if 'status' in metrics_df.columns:
                successful_pairs = metrics_df[metrics_df['status'] == 'Success']
                fallback_ransac = metrics_df[metrics_df['status'] == 'Fallback - RANSAC Only']
                fallback_motion = metrics_df[metrics_df['status'] == 'Fallback - Last Good Motion']
                failed_pairs = metrics_df[metrics_df['status'].str.startswith('Failed', na=False)] # Handle potential NaN status
                fallback_invert = metrics_df[metrics_df['status'] == 'Fallback - Matrix Inversion Error']

                f.write(f"- Successful ICP Registrations: {len(successful_pairs)}\n")
                f.write(f"- Fallback (RANSAC only): {len(fallback_ransac)}\n")
                f.write(f"- Fallback (Last Motion): {len(fallback_motion)}\n")
                f.write(f"- Fallback (Matrix Inversion): {len(fallback_invert)}\n")
                f.write(f"- Failed (Load/Preprocess/Normals): {len(failed_pairs)}\n")

                # Calculate averages only on successful results where metrics are valid
                valid_success = successful_pairs[(successful_pairs['icp_final_rmse'] >= 0) & (successful_pairs['ransac_init_rmse'] >= 0)]
                if not valid_success.empty:
                    avg_ransac_f = valid_success['ransac_init_fitness'].mean()
                    avg_ransac_e = valid_success['ransac_init_rmse'].mean()
                    avg_icp_f = valid_success['icp_final_fitness'].mean()
                    avg_icp_e = valid_success['icp_final_rmse'].mean()
                    f.write(f"\n  Average Metrics (Valid Successful Pairs Only):\n")
                    f.write(f"  - Avg. RANSAC Fitness (Initial Guess Eval): {avg_ransac_f:.4f}\n")
                    f.write(f"  - Avg. RANSAC RMSE (Initial Guess Eval): {avg_ransac_e:.4f}\n")
                    f.write(f"  - Avg. Final ICP Fitness: {avg_icp_f:.4f}\n")
                    f.write(f"  - Avg. Final ICP RMSE: {avg_icp_e:.4f}\n\n")
                else:
                    f.write("\n  No valid successful registrations to calculate average metrics.\n\n")
            else:
                f.write("  'status' column not found in metrics DataFrame.\n\n")

            f.write(f"Trajectory Information:\n")
            if trajectory_df is not None and not trajectory_df.empty:
                valid_traj_points = trajectory_df[['x', 'y', 'z']].dropna().values
                if len(valid_traj_points) > 1:
                    diffs = np.diff(valid_traj_points, axis=0)
                    segment_lengths = np.sqrt(np.sum(diffs**2, axis=1))
                    total_distance = np.sum(segment_lengths)
                    f.write(f"- Total estimated trajectory distance: {total_distance:.4f} meters\n")
                    # Add check for NaN before min/max
                    if trajectory_df['x'].notna().any():
                        f.write(f"- Bounding box (X): {trajectory_df['x'].min():.2f} to {trajectory_df['x'].max():.2f} m\n")
                    if trajectory_df['y'].notna().any():
                        f.write(f"- Bounding box (Y): {trajectory_df['y'].min():.2f} to {trajectory_df['y'].max():.2f} m\n")
                    if trajectory_df['z'].notna().any():
                        f.write(f"- Bounding box (Z): {trajectory_df['z'].min():.2f} to {trajectory_df['z'].max():.2f} m\n")
                else:
                    f.write("- Not enough valid points (>= 2) to calculate trajectory distance.\n")
            else:
                f.write("- Trajectory data is not available or empty.\n")

            f.write("\n--- End of Report ---")
        print(f"Summary report saved successfully.")
    except Exception as e:
        print(f"Error writing summary report: {e}")


# === Main Execution Block ===

def main():
    # --- Configuration ---
    # IMPORTANT: Update this path to your dataset location!
    dataset_dir = r"C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds" # Example path
    output_dir = "." # Directory to save outputs (CSV, PNG, PCD, TXT)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Registration Parameters for Part 4 (RANSAC + Point-to-Point ICP)
    preprocess_voxel_size = 0.05    # Voxel size for preprocessing (downsampling & normal radius base)
    icp_threshold = 0.1             # Max correspondence distance for P2P ICP (might need to be larger than for P2Plane)
    icp_max_iteration = 100         # Max iterations for P2P ICP refinement
    knn_normals = 30                # K-neighbors for normal estimation (needed for RANSAC FPFH)

    # Output File Names (Reflecting P2P ICP usage)
    trajectory_csv_file = os.path.join(output_dir, "estimated_trajectory_ransac_p2p.csv")
    metrics_csv_file = os.path.join(output_dir, "registration_metrics_ransac_p2p.csv")
    plot_3d_file = os.path.join(output_dir, "trajectory_plot_3d_ransac_p2p.png")
    plot_2d_file = os.path.join(output_dir, "trajectory_plot_2d_ransac_p2p.png")
    global_map_file = os.path.join(output_dir, "global_map_ransac_p2p.pcd")
    summary_report_file = os.path.join(output_dir, "registration_summary_ransac_p2p.txt")

    # Visualization/Saving Parameters for Combined Map
    map_downsample_voxel_size = 0.05   # Voxel size for downsampling the final map

    # --- End Configuration ---

    # 1. Get sorted list of point cloud files
    try:
        all_files = os.listdir(dataset_dir)
        pcd_files = sorted([f for f in all_files if f.startswith("pointcloud_") and f.endswith(".pcd")])
        if not pcd_files:
            print(f"Error: No PCD files matching 'pointcloud_*.pcd' found in '{dataset_dir}'.")
            return
        print(f"Found {len(pcd_files)} point cloud files in '{dataset_dir}'.")
    except FileNotFoundError:
        print(f"Error: Dataset directory not found: {dataset_dir}")
        return
    except Exception as e:
        print(f"Error accessing dataset directory: {e}")
        return

    # 2. Run Sequential Registration (using Point-to-Point ICP)
    print("\nStarting sequential registration process using RANSAC + Point-to-Point ICP...")
    params_info = {
        'dataset_dir': dataset_dir, 'num_files': len(pcd_files),
        'preprocess_voxel_size': preprocess_voxel_size, 'icp_threshold': icp_threshold,
        'icp_max_iteration': icp_max_iteration, 'knn': knn_normals
    }
    print(f"Parameters: Voxel={preprocess_voxel_size}, P2P_ICP Thresh={icp_threshold}, ICP Iter={icp_max_iteration}, Normals KNN={knn_normals}")

    registered_pcds_global, trajectory, metrics_df = register_all_point_clouds_p2p(
        dataset_dir, pcd_files,
        preprocess_voxel_size=preprocess_voxel_size,
        icp_threshold=icp_threshold,
        icp_max_iteration=icp_max_iteration,
        knn=knn_normals
    )

    # 3. Process Results
    if registered_pcds_global is None or trajectory is None or metrics_df is None:
        print("\nRegistration process failed or was aborted.")
        return

    print("\nRegistration processing complete!")
    valid_registered_pcds = [p for p in registered_pcds_global if p is not None and p.has_points()]
    print(f"Processed {len(pcd_files)} files, resulting in {len(valid_registered_pcds)} non-empty global clouds.")
    print(f"Trajectory contains {len(trajectory)} poses.")

    # 4. Save Metrics
    if metrics_df is not None and not metrics_df.empty:
        try:
            metrics_df.to_csv(metrics_csv_file, index=False)
            print(f"Saved registration metrics to '{metrics_csv_file}'")
        except Exception as e:
            print(f"Error saving metrics CSV: {e}")
    else:
        print("Metrics DataFrame is empty or None, nothing to save.")

    # 5. Save and Plot Trajectory
    trajectory_df = pd.DataFrame() # Initialize as empty
    if trajectory:
        trajectory_df = save_trajectory_to_csv(trajectory, trajectory_csv_file)
        # Check if DataFrame is valid AND contains data before plotting
        if trajectory_df is not None and not trajectory_df.empty and trajectory_df[['x', 'y', 'z']].notna().any().any():
            plot_title_suffix = " (RANSAC + P2P ICP)"
            plot_trajectory_3d(trajectory_df, filename=plot_3d_file, title_suffix=plot_title_suffix)
            plot_trajectory_2d(trajectory_df, filename=plot_2d_file, title_suffix=plot_title_suffix)
        elif trajectory_df is not None and trajectory_df.empty:
             print("Trajectory DataFrame created but is empty. Cannot plot.")
        elif trajectory_df is not None:
             print("Trajectory DataFrame contains only NaN values. Cannot plot.")
        else:
            print("Trajectory data could not be saved or processed into DataFrame for plotting.")
    else:
        print("Trajectory list is empty, cannot save or plot.")

    # 6. Create Summary Report
    create_summary_report(metrics_df if metrics_df is not None else pd.DataFrame(), # Pass empty df if None
                          trajectory_df if trajectory_df is not None else pd.DataFrame(),
                          params_info, summary_report_file)

    # 7. Visualize and Save Final Map
    if registered_pcds_global:
        visualize_and_save_combined_point_cloud(registered_pcds_global, trajectory,
                                                downsample_voxel_size=map_downsample_voxel_size,
                                                save_path=global_map_file,
                                                window_name="Global Map (RANSAC + P2P ICP)")
    else:
        print("No registered point clouds generated to visualize or save.")

    print("\n--- Script execution finished ---")


if __name__ == "__main__":
    # Ensure necessary imports for plotting are available
    try:
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
    except ImportError:
        print("Error: matplotlib is required for plotting. Please install it (`pip install matplotlib`)")
        # Optionally exit or disable plotting if matplotlib is missing
        # exit()
    main()

Found 100 point cloud files in 'C:/Users/aarya/Github_Projects/Striver_A2Z/cv_bonus/selected_pcds'.

Starting sequential registration process using RANSAC + Point-to-Point ICP...
Parameters: Voxel=0.05, P2P_ICP Thresh=0.1, ICP Iter=100, Normals KNN=30
Loading initial cloud 0: pointcloud_0000.pcd...
Preprocessing cloud 0 (voxel=0.05, knn=30)...
--------------------------------------------------
Processing Pair 0-1: [pointcloud_0000.pcd -> pointcloud_0004.pcd] (1/99)
--------------------------------------------------
Preprocessing cloud 1 (voxel=0.05, knn=30)...
Running RANSAC (Feature-based) for initial alignment...
Error: FPFH feature computation failed or resulted in incorrect number of features.
Source points: 9752, Source FPFH: Invalid
Target points: 9750, Target FPFH: Invalid
RANSAC Initial Guess Eval - Fitness: 0.0000, RMSE: -1.0000
Running Point-to-Point ICP for refinement...
P2P ICP Refinement Result - Fitness: 0.992514, RMSE: 0.020715
-------------------------------------------